In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import glob
import imageio
from keras.models import Model
from keras.models import load_model
from keras.layers import *
from keras import backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from src.data.dataset import FergZeroOne, Ferg
from src.trainer import PrivaterTrainer
from src.evaluater import PrivateTaskEvaluater, UtilityTaskEvaluater
from src.privater import Adae
from keras.callbacks import ModelCheckpoint
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Using TensorFlow backend.


In [3]:
ferg = Ferg.from_hdf5(select_people=[0, 1, 2, 3, 4, 5], transform=True)
ferg.show_info()

train data
x:(47401, 64, 64, 3)
y:(47401, 7)
p:(47401, 6)
test data
x:(8365, 64, 64, 3)
y:(8365, 7)
p:(8365, 6)


In [4]:
privater = Adae()
privater.load_weights('./adae_5000_weights.h5')
#trainer = PrivaterTrainer(evaluate_iter=300, d_iter=4)
evaluaters = [UtilityTaskEvaluater(verbose=True), PrivateTaskEvaluater(verbose=True)]
for evaluater in evaluaters:
    res = evaluater.evaluate(ferg, privater)
    print(res)
#historys = trainer.train(ferg, privater=privater, evaluaters=evaluaters)
#for history in historys:
#    print(history)

Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 2s 45us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9234 - val_acc: 0.1362
Epoch 2/30
47401/47401 [==============================] - 2s 41us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9234 - val_acc: 0.1362
Epoch 3/30
47401/47401 [==============================] - 2s 41us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9234 - val_acc: 0.1362
Epoch 4/30
47401/47401 [==============================] - 2s 41us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9234 - val_acc: 0.1362
Epoch 5/30
47401/47401 [==============================] - 2s 41us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9234 - val_acc: 0.1362
Epoch 6/30
47401/47401 [==============================] - 2s 41us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9234 - val_acc: 0.1362
Epoch 7/30
47401/47401 [==============================] - 2s 41us/step - loss: 14.0129 - acc: 0.1306 - val_loss: 13.9

In [ ]:
privater.g_train_model.save_weights('./adae_5000_weights.h5')

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255 - 0.5
x_test = x_test.astype('float32') / 255 - 0.5
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)
img_dim = x_train.shape[1]


z_dim = 256 # 隐变量维度
alpha = 0.5 # 全局互信息的loss比重
beta = 1.5 # 局部互信息的loss比重
gamma = 0.01 # 先验分布的loss比重


# 编码器（卷积与最大池化）
x_in = Input(shape=(img_dim, img_dim, 3))
x = x_in

for i in range(3):
    x = Conv2D(z_dim / 2**(2-i),
               kernel_size=(3,3),
               padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = MaxPooling2D((2, 2))(x)

feature_map = x # 截断到这里，认为到这里是feature_map（局部特征）
feature_map_encoder = Model(x_in, x)


for i in range(2):
    x = Conv2D(z_dim,
               kernel_size=(3,3),
               padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

x = GlobalMaxPooling2D()(x) # 全局特征

z_mean = Dense(z_dim)(x) # 均值，也就是最终输出的编码
z_log_var = Dense(z_dim)(x) # 方差，这里都是模仿VAE的


encoder = Model(x_in, z_mean) # 总的编码器就是输出z_mean


# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    u = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * u


# 重参数层，相当于给输入加入噪声
z_samples = Lambda(sampling)([z_mean, z_log_var])
prior_kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))


# shuffle层，打乱第一个轴
def shuffling(x):
    idxs = K.arange(0, K.shape(x)[0])
    idxs = K.tf.random_shuffle(idxs)
    return K.gather(x, idxs)


# 与随机采样的特征拼接（全局）
z_shuffle = Lambda(shuffling)(z_samples)
z_z_1 = Concatenate()([z_samples, z_samples])
z_z_2 = Concatenate()([z_samples, z_shuffle])

# 与随机采样的特征拼接（局部）
feature_map_shuffle = Lambda(shuffling)(feature_map)
z_samples_repeat = RepeatVector(4 * 4)(z_samples)
z_samples_map = Reshape((4, 4, z_dim))(z_samples_repeat)
z_f_1 = Concatenate()([z_samples_map, feature_map])
z_f_2 = Concatenate()([z_samples_map, feature_map_shuffle])


# 全局判别器
z_in = Input(shape=(z_dim*2,))
z = z_in
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)

GlobalDiscriminator = Model(z_in, z)

z_z_1_scores = GlobalDiscriminator(z_z_1)
z_z_2_scores = GlobalDiscriminator(z_z_2)
global_info_loss = - K.mean(K.log(z_z_1_scores + 1e-6) + K.log(1 - z_z_2_scores + 1e-6))


# 局部判别器
z_in = Input(shape=(None, None, z_dim*2))
z = z_in
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)

LocalDiscriminator = Model(z_in, z)

z_f_1_scores = LocalDiscriminator(z_f_1)
z_f_2_scores = LocalDiscriminator(z_f_2)
local_info_loss = - K.mean(K.log(z_f_1_scores + 1e-6) + K.log(1 - z_f_2_scores + 1e-6))

# 用来训练的模型
model_train = Model(x_in, [z_z_1_scores, z_z_2_scores, z_f_1_scores, z_f_2_scores])
model_train.add_loss(alpha * global_info_loss + beta * local_info_loss + gamma * prior_kl_loss)
model_train.compile(optimizer=Adam(1e-3))

model_train.fit(x_train, epochs=50, batch_size=64)
model_train.save_weights('total_model.cifar10.weights')


# 输出编码器的特征
zs = encoder.predict(x_train, verbose=True)
zs.mean() # 查看均值（简单观察先验分布有没有达到效果）
zs.std() # 查看方差（简单观察先验分布有没有达到效果）


# 随机选一张图片，输出最相近的图片
# 可以选用欧氏距离或者cos值
def sample_knn(path):
    n = 10
    topn = 10
    figure1 = np.zeros((img_dim*n, img_dim*topn, 3))
    figure2 = np.zeros((img_dim*n, img_dim*topn, 3))
    zs_ = zs / (zs**2).sum(1, keepdims=True)**0.5
    for i in range(n):
        one = np.random.choice(len(x_train))
        idxs = ((zs**2).sum(1) + (zs[one]**2).sum() - 2 * np.dot(zs, zs[one])).argsort()[:topn]
        for j,k in enumerate(idxs):
            digit = x_train[k]
            figure1[i*img_dim: (i+1)*img_dim,
                   j*img_dim: (j+1)*img_dim] = digit
        idxs = np.dot(zs_, zs_[one]).argsort()[-n:][::-1]
        for j,k in enumerate(idxs):
            digit = x_train[k]
            figure2[i*img_dim: (i+1)*img_dim,
                   j*img_dim: (j+1)*img_dim] = digit
    figure1 = (figure1 + 1) / 2 * 255
    figure1 = np.clip(figure1, 0, 255)
    figure2 = (figure2 + 1) / 2 * 255
    figure2 = np.clip(figure2, 0, 255)
    imageio.imwrite(path+'_l2.png', figure1)
    imageio.imwrite(path+'_cos.png', figure2)


sample_knn('test')

In [ ]:
iter 0: g_log=[18.863806, 0.4094513, 1.6087584, 0.375] d_log=[1.8245003, 0.34375]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 192us/step - loss: 1.8655 - acc: 0.2578 - val_loss: 1.7449 - val_acc: 0.3334
Epoch 2/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.5947 - acc: 0.4226 - val_loss: 1.4629 - val_acc: 0.4779
Epoch 3/30
47401/47401 [==============================] - 5s 101us/step - loss: 1.3290 - acc: 0.5717 - val_loss: 1.2274 - val_acc: 0.6340
Epoch 4/30
47401/47401 [==============================] - 5s 101us/step - loss: 1.1291 - acc: 0.6594 - val_loss: 1.0610 - val_acc: 0.7014
Epoch 5/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.9882 - acc: 0.7098 - val_loss: 0.9411 - val_acc: 0.7290
Epoch 6/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.8826 - acc: 0.7434 - val_loss: 0.8508 - val_acc: 0.7458
Epoch 7/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.8002 - acc: 0.7707 - val_loss: 0.7761 - val_acc: 0.7745
Epoch 8/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.7328 - acc: 0.7900 - val_loss: 0.7124 - val_acc: 0.7987
Epoch 9/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6762 - acc: 0.8058 - val_loss: 0.6640 - val_acc: 0.8091
Epoch 10/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6290 - acc: 0.8165 - val_loss: 0.6205 - val_acc: 0.8192
Epoch 11/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.5860 - acc: 0.8294 - val_loss: 0.5872 - val_acc: 0.8294
Epoch 12/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.5505 - acc: 0.8383 - val_loss: 0.5636 - val_acc: 0.8294
Epoch 13/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5192 - acc: 0.8477 - val_loss: 0.5150 - val_acc: 0.8526
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4905 - acc: 0.8560 - val_loss: 0.4976 - val_acc: 0.8487
Epoch 15/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4664 - acc: 0.8637 - val_loss: 0.4664 - val_acc: 0.8658
Epoch 16/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4427 - acc: 0.8707 - val_loss: 0.4470 - val_acc: 0.8718
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4222 - acc: 0.8763 - val_loss: 0.4258 - val_acc: 0.8747
Epoch 18/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4024 - acc: 0.8824 - val_loss: 0.4123 - val_acc: 0.8764
Epoch 19/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.3860 - acc: 0.8856 - val_loss: 0.3933 - val_acc: 0.8846
Epoch 20/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3693 - acc: 0.8903 - val_loss: 0.3758 - val_acc: 0.8886
Epoch 21/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3541 - acc: 0.8956 - val_loss: 0.3603 - val_acc: 0.8919
Epoch 22/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3402 - acc: 0.9002 - val_loss: 0.3473 - val_acc: 0.8930
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3273 - acc: 0.9027 - val_loss: 0.3345 - val_acc: 0.8996
Epoch 24/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3139 - acc: 0.9063 - val_loss: 0.3355 - val_acc: 0.8948
Epoch 25/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3019 - acc: 0.9104 - val_loss: 0.3169 - val_acc: 0.9016
Epoch 26/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2926 - acc: 0.9134 - val_loss: 0.3076 - val_acc: 0.9085
Epoch 27/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2816 - acc: 0.9173 - val_loss: 0.2938 - val_acc: 0.9109
Epoch 28/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2719 - acc: 0.9201 - val_loss: 0.2760 - val_acc: 0.9167
Epoch 29/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2628 - acc: 0.9222 - val_loss: 0.2718 - val_acc: 0.9167
Epoch 30/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2546 - acc: 0.9253 - val_loss: 0.2737 - val_acc: 0.9119
iter 0: eva_log=eva_acc=0.9166766288390219
iter 10: g_log=[3.3982558, 0.12049507, 2.6264977, 0.125] d_log=[2.2386699, 0.296875]
iter 20: g_log=[0.96892834, 0.08089556, 3.0758495, 0.09375] d_log=[2.951024, 0.03125]
iter 30: g_log=[1.6852508, 0.06269306, 1.4494023, 0.484375] d_log=[1.644057, 0.453125]
iter 40: g_log=[0.00928545, 0.046209905, 2.30121, 0.171875] d_log=[2.4344726, 0.1875]
iter 50: g_log=[0.38787985, 0.03811058, 1.5176492, 0.359375] d_log=[1.2969027, 0.453125]
iter 60: g_log=[-0.52623963, 0.032946993, 2.1735892, 0.1875] d_log=[2.4672396, 0.09375]
iter 70: g_log=[0.14719033, 0.03371054, 1.5383366, 0.4375] d_log=[1.712012, 0.3125]
iter 80: g_log=[0.14916813, 0.030557334, 1.3786986, 0.21875] d_log=[1.4535918, 0.171875]
iter 90: g_log=[0.083788395, 0.030656047, 1.449014, 0.265625] d_log=[1.5404074, 0.234375]
iter 100: g_log=[-0.68702173, 0.025428634, 1.9584534, 0.140625] d_log=[1.916536, 0.234375]
iter 110: g_log=[-0.7243123, 0.021587376, 1.8036811, 0.40625] d_log=[1.7043812, 0.296875]
iter 120: g_log=[-0.15587795, 0.025526993, 1.4322276, 0.484375] d_log=[1.529388, 0.375]
iter 130: g_log=[-0.3048954, 0.019876182, 1.2987045, 0.484375] d_log=[1.1836847, 0.46875]
iter 140: g_log=[-0.5297551, 0.023677673, 1.7136388, 0.421875] d_log=[1.9188493, 0.21875]
iter 150: g_log=[-0.8021697, 0.023411712, 1.9727553, 0.15625] d_log=[2.1416726, 0.046875]
iter 160: g_log=[1.4083985, 0.044116713, 0.7974371, 1.0] d_log=[0.81980693, 0.953125]
iter 170: g_log=[1.0919592, 0.037937626, 0.80492216, 0.953125] d_log=[0.7262408, 0.953125]
iter 180: g_log=[0.4847169, 0.03328002, 1.1792841, 0.4375] d_log=[1.3085767, 0.234375]
iter 190: g_log=[0.06331146, 0.031276174, 1.5004972, 0.359375] d_log=[1.589216, 0.171875]
iter 200: g_log=[-0.09568727, 0.028270092, 1.5091919, 0.40625] d_log=[1.5735695, 0.4375]
iter 210: g_log=[-0.898823, 0.02562489, 2.1800675, 0.359375] d_log=[1.882893, 0.40625]
iter 220: g_log=[-1.5139763, 0.023237925, 2.6758726, 0.0] d_log=[2.708077, 0.0]
iter 230: g_log=[-0.60923326, 0.01830137, 1.5243018, 0.359375] d_log=[1.5051768, 0.546875]
iter 240: g_log=[-0.87998396, 0.01971734, 1.8658509, 0.15625] d_log=[1.7938201, 0.21875]
iter 250: g_log=[-1.046196, 0.016381826, 1.8652873, 0.109375] d_log=[1.7968907, 0.1875]
iter 260: g_log=[-0.4864685, 0.015763236, 1.2746303, 0.46875] d_log=[1.2400105, 0.5625]
iter 270: g_log=[-0.33882236, 0.014700022, 1.0738235, 0.46875] d_log=[1.1843005, 0.453125]
iter 280: g_log=[-0.82154953, 0.01779794, 1.7114465, 0.265625] d_log=[1.8502069, 0.28125]
iter 290: g_log=[-0.8977028, 0.017464086, 1.7709072, 0.28125] d_log=[1.7573545, 0.28125]
iter 300: g_log=[-0.60696524, 0.018171042, 1.5155174, 0.3125] d_log=[1.5537935, 0.21875]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 196us/step - loss: 1.8950 - acc: 0.2312 - val_loss: 1.7899 - val_acc: 0.3591
Epoch 2/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.6797 - acc: 0.3437 - val_loss: 1.5434 - val_acc: 0.4485
Epoch 3/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.5209 - acc: 0.4094 - val_loss: 1.4012 - val_acc: 0.5621
Epoch 4/30
47401/47401 [==============================] - 5s 101us/step - loss: 1.3947 - acc: 0.4645 - val_loss: 1.5032 - val_acc: 0.3913
Epoch 5/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.2643 - acc: 0.5279 - val_loss: 1.3698 - val_acc: 0.4773
Epoch 6/30
47401/47401 [==============================] - 5s 101us/step - loss: 1.1461 - acc: 0.5835 - val_loss: 1.2013 - val_acc: 0.5441
Epoch 7/30
47401/47401 [==============================] - 5s 101us/step - loss: 1.0440 - acc: 0.6285 - val_loss: 0.9884 - val_acc: 0.6432
Epoch 8/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.9604 - acc: 0.6634 - val_loss: 0.8494 - val_acc: 0.7339
Epoch 9/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.8797 - acc: 0.6993 - val_loss: 0.9188 - val_acc: 0.6453
Epoch 10/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.8154 - acc: 0.7201 - val_loss: 0.7589 - val_acc: 0.7542
Epoch 11/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.7492 - acc: 0.7482 - val_loss: 0.7285 - val_acc: 0.7782
Epoch 12/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.7058 - acc: 0.7592 - val_loss: 0.7805 - val_acc: 0.7262
Epoch 13/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6574 - acc: 0.7778 - val_loss: 0.6096 - val_acc: 0.8169
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6113 - acc: 0.7933 - val_loss: 0.6068 - val_acc: 0.7837
Epoch 15/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.6000 - acc: 0.7923 - val_loss: 0.5447 - val_acc: 0.8440
Epoch 16/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.5476 - acc: 0.8149 - val_loss: 0.4954 - val_acc: 0.8634
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5266 - acc: 0.8213 - val_loss: 0.6419 - val_acc: 0.7733
Epoch 18/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5088 - acc: 0.8257 - val_loss: 0.4522 - val_acc: 0.8493
Epoch 19/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4793 - acc: 0.8368 - val_loss: 0.4039 - val_acc: 0.8796
Epoch 20/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4612 - acc: 0.8420 - val_loss: 0.4052 - val_acc: 0.8844
Epoch 21/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4546 - acc: 0.8427 - val_loss: 0.5121 - val_acc: 0.8081
Epoch 22/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4448 - acc: 0.8431 - val_loss: 0.6244 - val_acc: 0.7604
Epoch 23/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4071 - acc: 0.8599 - val_loss: 0.4320 - val_acc: 0.8375
Epoch 24/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.4000 - acc: 0.8595 - val_loss: 0.3574 - val_acc: 0.9002
Epoch 25/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3888 - acc: 0.8644 - val_loss: 0.4227 - val_acc: 0.8496
Epoch 26/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3913 - acc: 0.8623 - val_loss: 0.3887 - val_acc: 0.8641
Epoch 27/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3473 - acc: 0.8814 - val_loss: 0.3540 - val_acc: 0.8635
Epoch 28/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3582 - acc: 0.8745 - val_loss: 0.3397 - val_acc: 0.8735
Epoch 29/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3582 - acc: 0.8710 - val_loss: 0.3379 - val_acc: 0.8860
Epoch 30/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.3384 - acc: 0.8806 - val_loss: 0.3179 - val_acc: 0.8954
iter 300: eva_log=eva_acc=0.881416003899379
iter 310: g_log=[-1.1248802, 0.016239617, 1.936861, 0.1875] d_log=[2.0841212, 0.1875]
iter 320: g_log=[-1.0183154, 0.01572787, 1.804709, 0.15625] d_log=[1.8968699, 0.203125]
iter 330: g_log=[-0.55284864, 0.01417373, 1.2615352, 0.578125] d_log=[1.5007651, 0.484375]
iter 340: g_log=[-0.7503449, 0.015551002, 1.527895, 0.203125] d_log=[1.6093639, 0.109375]
iter 350: g_log=[-1.7283466, 0.016089868, 2.53284, 0.0] d_log=[2.7891107, 0.0]
iter 360: g_log=[-0.32979655, 0.01394202, 1.0268975, 0.859375] d_log=[1.0912781, 0.703125]
iter 370: g_log=[-0.5368377, 0.012794204, 1.1765479, 0.625] d_log=[1.2366745, 0.4375]
iter 380: g_log=[-2.0090013, 0.01414443, 2.7162228, 0.0625] d_log=[2.6344085, 0.0]
iter 390: g_log=[-1.1729436, 0.012679385, 1.8069129, 0.0] d_log=[1.833317, 0.0]
iter 400: g_log=[-0.8149297, 0.014711595, 1.5505095, 0.28125] d_log=[1.5507208, 0.34375]
iter 410: g_log=[-0.869873, 0.015044199, 1.622083, 0.28125] d_log=[1.6592879, 0.390625]
iter 420: g_log=[-1.2173749, 0.014111736, 1.9229617, 0.15625] d_log=[1.9207829, 0.328125]
iter 430: g_log=[-1.1540866, 0.013866505, 1.8474119, 0.15625] d_log=[1.8530375, 0.1875]
iter 440: g_log=[-0.9283745, 0.013565045, 1.6066267, 0.125] d_log=[1.6499391, 0.0]
iter 450: g_log=[-1.0045159, 0.01237037, 1.6230345, 0.234375] d_log=[1.6618168, 0.1875]
iter 460: g_log=[-1.0241082, 0.016498083, 1.8490124, 0.046875] d_log=[1.7913202, 0.328125]
iter 470: g_log=[-1.227359, 0.012603687, 1.8575435, 0.265625] d_log=[1.801019, 0.21875]
iter 480: g_log=[-1.0438379, 0.01383466, 1.7355709, 0.140625] d_log=[1.7296097, 0.015625]
iter 490: g_log=[-1.0321145, 0.013047958, 1.6845124, 0.421875] d_log=[1.6664264, 0.4375]
iter 500: g_log=[-1.0557613, 0.011206262, 1.6160744, 0.59375] d_log=[1.5993416, 0.609375]
iter 510: g_log=[-1.0295873, 0.012179905, 1.6385825, 0.1875] d_log=[1.7851655, 0.140625]
iter 520: g_log=[-1.0990264, 0.011759549, 1.6870039, 0.25] d_log=[1.7605222, 0.171875]
iter 530: g_log=[-1.1459267, 0.012262655, 1.7590594, 0.234375] d_log=[1.6927874, 0.296875]
iter 540: g_log=[-1.0369909, 0.013106438, 1.6923127, 0.328125] d_log=[1.6867235, 0.265625]
iter 550: g_log=[-0.9854065, 0.012493713, 1.6100922, 0.421875] d_log=[1.6113427, 0.296875]
iter 560: g_log=[-1.164073, 0.013319511, 1.8300486, 0.140625] d_log=[1.8055184, 0.203125]
iter 570: g_log=[-1.087718, 0.012657009, 1.7205684, 0.203125] d_log=[1.854171, 0.125]
iter 580: g_log=[-0.9394036, 0.012359684, 1.5573878, 0.203125] d_log=[1.5740428, 0.21875]
iter 590: g_log=[-0.9320248, 0.012852863, 1.5746679, 0.25] d_log=[1.5849004, 0.40625]
iter 600: g_log=[-1.0952904, 0.012428712, 1.7167261, 0.234375] d_log=[1.7845664, 0.25]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 193us/step - loss: 2.0232 - acc: 0.1965 - val_loss: 1.8545 - val_acc: 0.2785
Epoch 2/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.8229 - acc: 0.2788 - val_loss: 1.8446 - val_acc: 0.2464
Epoch 3/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.6676 - acc: 0.3703 - val_loss: 1.5613 - val_acc: 0.3843
Epoch 4/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.5018 - acc: 0.4352 - val_loss: 1.4177 - val_acc: 0.4634
Epoch 5/30
47401/47401 [==============================] - 5s 101us/step - loss: 1.3551 - acc: 0.4952 - val_loss: 1.2807 - val_acc: 0.5602
Epoch 6/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.2437 - acc: 0.5343 - val_loss: 1.1777 - val_acc: 0.5354
Epoch 7/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.1566 - acc: 0.5723 - val_loss: 1.2107 - val_acc: 0.5225
Epoch 8/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.0879 - acc: 0.5944 - val_loss: 1.1012 - val_acc: 0.5071
Epoch 9/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.0169 - acc: 0.6214 - val_loss: 0.9468 - val_acc: 0.6582
Epoch 10/30
47401/47401 [==============================] - 5s 100us/step - loss: 0.9714 - acc: 0.6342 - val_loss: 1.0377 - val_acc: 0.5817
Epoch 11/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.9074 - acc: 0.6589 - val_loss: 0.9177 - val_acc: 0.6504
Epoch 12/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.8686 - acc: 0.6735 - val_loss: 0.8797 - val_acc: 0.6491
Epoch 13/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.8140 - acc: 0.7013 - val_loss: 0.8131 - val_acc: 0.6652
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.7874 - acc: 0.7071 - val_loss: 0.7101 - val_acc: 0.7512
Epoch 15/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.7396 - acc: 0.7248 - val_loss: 0.7576 - val_acc: 0.6887
Epoch 16/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.7111 - acc: 0.7404 - val_loss: 0.6203 - val_acc: 0.8073
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.6791 - acc: 0.7557 - val_loss: 0.6357 - val_acc: 0.7888
Epoch 18/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.6565 - acc: 0.7606 - val_loss: 0.6750 - val_acc: 0.7368
Epoch 19/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6423 - acc: 0.7656 - val_loss: 0.6778 - val_acc: 0.7443
Epoch 20/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5945 - acc: 0.7906 - val_loss: 0.5711 - val_acc: 0.7988
Epoch 21/30
47401/47401 [==============================] - 5s 99us/step - loss: 0.6017 - acc: 0.7792 - val_loss: 0.5871 - val_acc: 0.7944
Epoch 22/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5717 - acc: 0.7923 - val_loss: 0.5357 - val_acc: 0.8023
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.5509 - acc: 0.8033 - val_loss: 0.4689 - val_acc: 0.8353
Epoch 24/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5278 - acc: 0.8131 - val_loss: 0.5358 - val_acc: 0.8112
Epoch 25/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5302 - acc: 0.8089 - val_loss: 0.6859 - val_acc: 0.7485
Epoch 26/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5131 - acc: 0.8147 - val_loss: 0.4364 - val_acc: 0.8783
Epoch 27/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.5165 - acc: 0.8126 - val_loss: 0.4705 - val_acc: 0.8250
Epoch 28/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5076 - acc: 0.8197 - val_loss: 0.4682 - val_acc: 0.8207
Epoch 29/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4694 - acc: 0.8363 - val_loss: 0.3871 - val_acc: 0.8753
Epoch 30/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4630 - acc: 0.8362 - val_loss: 0.4425 - val_acc: 0.8328
iter 600: eva_log=eva_acc=0.836332566829772
iter 610: g_log=[-1.101192, 0.011366917, 1.6695378, 0.40625] d_log=[1.7325095, 0.265625]
iter 620: g_log=[-1.0548763, 0.012062551, 1.6580039, 0.390625] d_log=[1.6023757, 0.4375]
iter 630: g_log=[-1.0975146, 0.013156479, 1.7553387, 0.125] d_log=[1.7547133, 0.125]
iter 640: g_log=[-1.1597991, 0.011489911, 1.7342947, 0.375] d_log=[1.664037, 0.203125]
iter 650: g_log=[-1.1956314, 0.010716706, 1.7314667, 0.28125] d_log=[1.7146404, 0.234375]
iter 660: g_log=[-0.9725227, 0.0108331265, 1.514179, 0.359375] d_log=[1.5338972, 0.359375]
iter 670: g_log=[-1.044559, 0.010235639, 1.5563409, 0.375] d_log=[1.5521032, 0.4375]
iter 680: g_log=[-1.5390835, 0.011012945, 2.0897307, 0.203125] d_log=[2.0004606, 0.25]
iter 690: g_log=[-1.1500547, 0.011371179, 1.7186136, 0.34375] d_log=[1.5968167, 0.5625]
iter 700: g_log=[-0.97982675, 0.011354333, 1.5475434, 0.265625] d_log=[1.6350859, 0.171875]
iter 710: g_log=[-1.1628106, 0.011931556, 1.7593883, 0.234375] d_log=[1.704873, 0.28125]
iter 720: g_log=[-1.2682141, 0.010463877, 1.791408, 0.140625] d_log=[1.7425519, 0.234375]
iter 730: g_log=[-1.2145379, 0.010795426, 1.7543092, 0.203125] d_log=[1.7493265, 0.171875]
iter 740: g_log=[-1.1137991, 0.012919394, 1.7597688, 0.1875] d_log=[1.7417552, 0.203125]
iter 750: g_log=[-0.9710878, 0.015662245, 1.7542001, 0.109375] d_log=[1.7019866, 0.203125]
iter 760: g_log=[-1.1410396, 0.0121446345, 1.7482713, 0.421875] d_log=[1.6234552, 0.453125]
iter 770: g_log=[-1.2271674, 0.01007732, 1.7310334, 0.234375] d_log=[1.7604561, 0.203125]
iter 780: g_log=[-1.2011727, 0.010700762, 1.7362108, 0.234375] d_log=[1.8286381, 0.140625]
iter 790: g_log=[-1.3395681, 0.009498185, 1.8144773, 0.203125] d_log=[1.7948132, 0.09375]
iter 800: g_log=[-1.2502152, 0.010027222, 1.7515763, 0.171875] d_log=[1.7472353, 0.078125]
iter 810: g_log=[-1.0291588, 0.0108153615, 1.569927, 0.390625] d_log=[1.5834607, 0.390625]
iter 820: g_log=[-1.0320455, 0.009780364, 1.5210637, 0.5] d_log=[1.6721681, 0.390625]
iter 830: g_log=[-1.1891581, 0.011296489, 1.7539825, 0.1875] d_log=[1.7605231, 0.140625]
iter 840: g_log=[-1.2025061, 0.0101001775, 1.7075149, 0.21875] d_log=[1.6826718, 0.265625]
iter 850: g_log=[-1.1444455, 0.010560796, 1.6724854, 0.3125] d_log=[1.7205629, 0.21875]
iter 860: g_log=[-1.1559565, 0.0104898885, 1.6804509, 0.4375] d_log=[1.7057137, 0.390625]
iter 870: g_log=[-1.1255434, 0.011240509, 1.6875689, 0.1875] d_log=[1.7053167, 0.109375]
iter 880: g_log=[-1.1532979, 0.009691535, 1.6378746, 0.40625] d_log=[1.7205715, 0.1875]
iter 890: g_log=[-1.3066114, 0.008622148, 1.7377188, 0.203125] d_log=[1.7796575, 0.15625]
iter 900: g_log=[-1.3180163, 0.008940991, 1.7650659, 0.0] d_log=[1.7819477, 0.0]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 196us/step - loss: 1.8461 - acc: 0.2863 - val_loss: 1.5770 - val_acc: 0.3677
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.4373 - acc: 0.4715 - val_loss: 1.2944 - val_acc: 0.5547
Epoch 3/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.1564 - acc: 0.5952 - val_loss: 1.0265 - val_acc: 0.6838
Epoch 4/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.9614 - acc: 0.6737 - val_loss: 0.9126 - val_acc: 0.6583
Epoch 5/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.8162 - acc: 0.7296 - val_loss: 0.7479 - val_acc: 0.7482
Epoch 6/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.7008 - acc: 0.7802 - val_loss: 0.6974 - val_acc: 0.7589
Epoch 7/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.6283 - acc: 0.7990 - val_loss: 0.5523 - val_acc: 0.8434
Epoch 8/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5494 - acc: 0.8281 - val_loss: 0.5115 - val_acc: 0.8398
Epoch 9/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4951 - acc: 0.8438 - val_loss: 0.5351 - val_acc: 0.8250
Epoch 10/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4468 - acc: 0.8591 - val_loss: 0.4562 - val_acc: 0.8515
Epoch 11/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4094 - acc: 0.8698 - val_loss: 0.3563 - val_acc: 0.9062
Epoch 12/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3798 - acc: 0.8789 - val_loss: 0.3380 - val_acc: 0.9025
Epoch 13/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3480 - acc: 0.8871 - val_loss: 0.2993 - val_acc: 0.9069
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3329 - acc: 0.8890 - val_loss: 0.2630 - val_acc: 0.9340
Epoch 15/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3210 - acc: 0.8910 - val_loss: 0.3095 - val_acc: 0.8980
Epoch 16/30
47401/47401 [==============================] - 5s 100us/step - loss: 0.3088 - acc: 0.8951 - val_loss: 0.2928 - val_acc: 0.8924
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2848 - acc: 0.9059 - val_loss: 0.2330 - val_acc: 0.9335
Epoch 18/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2868 - acc: 0.8980 - val_loss: 0.3129 - val_acc: 0.8781
Epoch 19/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2659 - acc: 0.9087 - val_loss: 0.2467 - val_acc: 0.9133
Epoch 20/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2604 - acc: 0.9110 - val_loss: 0.2041 - val_acc: 0.9456
Epoch 21/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2513 - acc: 0.9143 - val_loss: 0.2155 - val_acc: 0.9216
Epoch 22/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2301 - acc: 0.9220 - val_loss: 0.2551 - val_acc: 0.9112
Epoch 23/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2320 - acc: 0.9180 - val_loss: 0.1986 - val_acc: 0.9267
Epoch 24/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2335 - acc: 0.9186 - val_loss: 0.2227 - val_acc: 0.8992
Epoch 25/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2232 - acc: 0.9231 - val_loss: 0.1916 - val_acc: 0.9334
Epoch 26/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2010 - acc: 0.9320 - val_loss: 0.1655 - val_acc: 0.9431
Epoch 27/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2189 - acc: 0.9216 - val_loss: 0.1917 - val_acc: 0.9259
Epoch 28/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.1915 - acc: 0.9345 - val_loss: 0.1591 - val_acc: 0.9418
Epoch 29/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2030 - acc: 0.9257 - val_loss: 0.2470 - val_acc: 0.9099
Epoch 30/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.1916 - acc: 0.9331 - val_loss: 0.2236 - val_acc: 0.8961
iter 900: eva_log=eva_acc=0.9344950528671057
iter 910: g_log=[-1.4093673, 0.008286366, 1.8236856, 0.0] d_log=[1.8208687, 0.203125]
iter 920: g_log=[-1.3450114, 0.008781016, 1.7840621, 0.0] d_log=[1.7850883, 0.0]
iter 930: g_log=[-1.3604243, 0.007255417, 1.7231951, 0.3125] d_log=[1.7463208, 0.140625]
iter 940: g_log=[-1.1142932, 0.008205779, 1.5245821, 0.578125] d_log=[1.5378948, 0.578125]
iter 950: g_log=[-1.0356746, 0.007835049, 1.427427, 0.59375] d_log=[1.4097214, 0.59375]
iter 960: g_log=[-1.3788011, 0.0075863833, 1.7581203, 0.359375] d_log=[1.80853, 0.25]
iter 970: g_log=[-1.6669343, 0.007393145, 2.0365915, 0.0] d_log=[2.0746012, 0.0]
iter 980: g_log=[-1.3537164, 0.007832655, 1.745349, 0.234375] d_log=[1.7745798, 0.203125]
iter 990: g_log=[-1.2190684, 0.009048464, 1.6714916, 0.421875] d_log=[1.7254915, 0.171875]
iter 1000: g_log=[-1.2565403, 0.006973081, 1.6051943, 0.5] d_log=[1.6826289, 0.34375]
iter 1010: g_log=[-1.2178426, 0.0072304206, 1.5793636, 0.4375] d_log=[1.5422299, 0.421875]
iter 1020: g_log=[-1.2811738, 0.007815938, 1.6719707, 0.328125] d_log=[1.6246891, 0.390625]
iter 1030: g_log=[-1.3032014, 0.008644536, 1.7354282, 0.21875] d_log=[1.8035316, 0.140625]
iter 1040: g_log=[-1.2898524, 0.0096337255, 1.7715386, 0.15625] d_log=[1.7734759, 0.15625]
iter 1050: g_log=[-1.3437572, 0.007311307, 1.7093225, 0.234375] d_log=[1.6818962, 0.25]
iter 1060: g_log=[-1.3498347, 0.007694725, 1.734571, 0.203125] d_log=[1.6593623, 0.3125]
iter 1070: g_log=[-1.3103732, 0.0075720637, 1.6889764, 0.296875] d_log=[1.7793719, 0.1875]
iter 1080: g_log=[-1.3966378, 0.007111375, 1.7522066, 0.234375] d_log=[1.7680768, 0.234375]
iter 1090: g_log=[-1.4293681, 0.0063907653, 1.7489064, 0.25] d_log=[1.7548587, 0.21875]
iter 1100: g_log=[-1.316169, 0.008829785, 1.7576582, 0.265625] d_log=[1.782139, 0.171875]
iter 1110: g_log=[-1.4338613, 0.007313901, 1.7995564, 0.1875] d_log=[1.7632767, 0.15625]
iter 1120: g_log=[-1.4058721, 0.0070934305, 1.7605436, 0.1875] d_log=[1.8111341, 0.265625]
iter 1130: g_log=[-1.4233408, 0.0070331306, 1.7749974, 0.1875] d_log=[1.7379287, 0.1875]
iter 1140: g_log=[-1.3164842, 0.007991085, 1.7160385, 0.25] d_log=[1.7181801, 0.203125]
iter 1150: g_log=[-1.2356229, 0.008923959, 1.6818209, 0.296875] d_log=[1.7262688, 0.25]
iter 1160: g_log=[-1.4163527, 0.006430649, 1.7378852, 0.203125] d_log=[1.7685208, 0.140625]
iter 1170: g_log=[-1.4794195, 0.006646581, 1.8117485, 0.140625] d_log=[1.7470016, 0.171875]
iter 1180: g_log=[-1.3969436, 0.0063508786, 1.7144876, 0.4375] d_log=[1.7485945, 0.390625]
iter 1190: g_log=[-1.4187311, 0.0073010754, 1.7837849, 0.15625] d_log=[1.74009, 0.25]
iter 1200: g_log=[-1.435776, 0.006611966, 1.7663743, 0.1875] d_log=[1.7735142, 0.171875]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 199us/step - loss: 1.8462 - acc: 0.2886 - val_loss: 1.6217 - val_acc: 0.3608
Epoch 2/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.4160 - acc: 0.4719 - val_loss: 1.2176 - val_acc: 0.6285
Epoch 3/30
47401/47401 [==============================] - 5s 105us/step - loss: 1.1626 - acc: 0.5778 - val_loss: 1.0133 - val_acc: 0.6659
Epoch 4/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.0078 - acc: 0.6346 - val_loss: 0.8866 - val_acc: 0.7522
Epoch 5/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.8932 - acc: 0.6844 - val_loss: 0.9241 - val_acc: 0.6324
Epoch 6/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.8071 - acc: 0.7104 - val_loss: 0.7432 - val_acc: 0.7403
Epoch 7/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.7305 - acc: 0.7398 - val_loss: 0.6548 - val_acc: 0.7788
Epoch 8/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.6605 - acc: 0.7647 - val_loss: 0.5857 - val_acc: 0.8049
Epoch 9/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.6172 - acc: 0.7787 - val_loss: 0.6189 - val_acc: 0.7695
Epoch 10/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.5741 - acc: 0.7931 - val_loss: 0.5487 - val_acc: 0.7921
Epoch 11/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5450 - acc: 0.8018 - val_loss: 0.5140 - val_acc: 0.8200
Epoch 12/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5157 - acc: 0.8109 - val_loss: 0.4509 - val_acc: 0.8549
Epoch 13/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4756 - acc: 0.8261 - val_loss: 0.4915 - val_acc: 0.8203
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4606 - acc: 0.8312 - val_loss: 0.4125 - val_acc: 0.8485
Epoch 15/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4202 - acc: 0.8472 - val_loss: 0.3454 - val_acc: 0.9004
Epoch 16/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4085 - acc: 0.8532 - val_loss: 0.3353 - val_acc: 0.9035
Epoch 17/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3820 - acc: 0.8613 - val_loss: 0.3438 - val_acc: 0.8827
Epoch 18/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3813 - acc: 0.8595 - val_loss: 0.3028 - val_acc: 0.9144
Epoch 19/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.3609 - acc: 0.8691 - val_loss: 0.3269 - val_acc: 0.8767
Epoch 20/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3277 - acc: 0.8813 - val_loss: 0.4181 - val_acc: 0.8409
Epoch 21/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3312 - acc: 0.8801 - val_loss: 0.2688 - val_acc: 0.9157
Epoch 22/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3010 - acc: 0.8927 - val_loss: 0.3022 - val_acc: 0.8910
Epoch 23/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2989 - acc: 0.8905 - val_loss: 0.2933 - val_acc: 0.9044
Epoch 24/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2766 - acc: 0.8998 - val_loss: 0.2601 - val_acc: 0.9228
Epoch 25/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2980 - acc: 0.8893 - val_loss: 0.3424 - val_acc: 0.8712
Epoch 26/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2627 - acc: 0.9047 - val_loss: 0.2668 - val_acc: 0.8857
Epoch 27/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2598 - acc: 0.9053 - val_loss: 0.2565 - val_acc: 0.8942
Epoch 28/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2533 - acc: 0.9057 - val_loss: 0.2755 - val_acc: 0.9041
Epoch 29/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2346 - acc: 0.9156 - val_loss: 0.3027 - val_acc: 0.8722
Epoch 30/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2515 - acc: 0.9063 - val_loss: 0.2067 - val_acc: 0.9286
iter 1200: eva_log=eva_acc=0.9155924980372473
iter 1210: g_log=[-1.5096924, 0.0061390046, 1.8166426, 0.15625] d_log=[1.752598, 0.171875]
iter 1220: g_log=[-1.4808915, 0.0058888253, 1.7753327, 0.203125] d_log=[1.7671027, 0.21875]
iter 1230: g_log=[-1.4851812, 0.0062080366, 1.795583, 0.03125] d_log=[1.7628276, 0.015625]
iter 1240: g_log=[-1.4733188, 0.0062996633, 1.788302, 0.109375] d_log=[1.7747943, 0.15625]
iter 1250: g_log=[-1.448312, 0.0065240054, 1.7745123, 0.234375] d_log=[1.7081231, 0.25]
iter 1260: g_log=[-1.3092574, 0.008508408, 1.7346778, 0.1875] d_log=[1.716198, 0.25]
iter 1270: g_log=[-1.4143907, 0.0056911893, 1.6989502, 0.21875] d_log=[1.7113162, 0.15625]
iter 1280: g_log=[-1.4021354, 0.0066450937, 1.73439, 0.40625] d_log=[1.7233584, 0.46875]
iter 1290: g_log=[-1.4485692, 0.006833174, 1.7902279, 0.171875] d_log=[1.731353, 0.265625]
iter 1300: g_log=[-1.4009595, 0.008333115, 1.8176153, 0.125] d_log=[1.7698103, 0.234375]
iter 1310: g_log=[-1.3801045, 0.0070539056, 1.7327998, 0.3125] d_log=[1.7706344, 0.1875]
iter 1320: g_log=[-1.4764161, 0.0061008325, 1.7814578, 0.15625] d_log=[1.7602373, 0.15625]
iter 1330: g_log=[-1.2952963, 0.009682453, 1.779419, 0.15625] d_log=[1.7257619, 0.421875]
iter 1340: g_log=[-1.441039, 0.0064734817, 1.764713, 0.171875] d_log=[1.7429814, 0.203125]
iter 1350: g_log=[-1.3940554, 0.0059655774, 1.6923343, 0.296875] d_log=[1.6918126, 0.1875]
iter 1360: g_log=[-1.3898609, 0.007009453, 1.7403336, 0.375] d_log=[1.7277716, 0.28125]
iter 1370: g_log=[-1.4554341, 0.005604001, 1.7356342, 0.390625] d_log=[1.7428095, 0.375]
iter 1380: g_log=[-1.4682105, 0.0059366804, 1.7650445, 0.1875] d_log=[1.7465537, 0.25]
iter 1390: g_log=[-1.494239, 0.0058547854, 1.7869782, 0.15625] d_log=[1.7769988, 0.171875]
iter 1400: g_log=[-1.3795538, 0.008241411, 1.7916244, 0.109375] d_log=[1.7803257, 0.1875]
iter 1410: g_log=[-1.3840154, 0.0073261587, 1.7503234, 0.25] d_log=[1.7762465, 0.203125]
iter 1420: g_log=[-1.4344876, 0.006070513, 1.7380133, 0.21875] d_log=[1.7438059, 0.15625]
iter 1430: g_log=[-1.4496658, 0.006566721, 1.7780018, 0.15625] d_log=[1.718235, 0.234375]
iter 1440: g_log=[-1.5055916, 0.005209948, 1.766089, 0.171875] d_log=[1.7712704, 0.15625]
iter 1450: g_log=[-1.4709439, 0.0056177615, 1.751832, 0.203125] d_log=[1.7360698, 0.15625]
iter 1460: g_log=[-1.4844882, 0.005639032, 1.7664399, 0.390625] d_log=[1.6800071, 0.46875]
iter 1470: g_log=[-1.4719546, 0.0056955907, 1.7567341, 0.140625] d_log=[1.761361, 0.171875]
iter 1480: g_log=[-1.4533637, 0.0064529255, 1.7760099, 0.1875] d_log=[1.7795277, 0.21875]
iter 1490: g_log=[-1.4511088, 0.0058332914, 1.7427734, 0.203125] d_log=[1.7597551, 0.171875]
iter 1500: g_log=[-1.4857867, 0.0055864523, 1.7651093, 0.25] d_log=[1.7784302, 0.21875]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 198us/step - loss: 1.9224 - acc: 0.2379 - val_loss: 1.8024 - val_acc: 0.2395
Epoch 2/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.6086 - acc: 0.4013 - val_loss: 1.4750 - val_acc: 0.4729
Epoch 3/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.3626 - acc: 0.5092 - val_loss: 1.1915 - val_acc: 0.6494
Epoch 4/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.1641 - acc: 0.5937 - val_loss: 1.1796 - val_acc: 0.5609
Epoch 5/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.0031 - acc: 0.6584 - val_loss: 0.9454 - val_acc: 0.6758
Epoch 6/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.8670 - acc: 0.7098 - val_loss: 0.8170 - val_acc: 0.7426
Epoch 7/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.7931 - acc: 0.7290 - val_loss: 0.7151 - val_acc: 0.7608
Epoch 8/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.7010 - acc: 0.7661 - val_loss: 0.6694 - val_acc: 0.8169
Epoch 9/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.6434 - acc: 0.7840 - val_loss: 0.5912 - val_acc: 0.7895
Epoch 10/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5992 - acc: 0.7943 - val_loss: 0.5385 - val_acc: 0.8281
Epoch 11/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5468 - acc: 0.8149 - val_loss: 0.5568 - val_acc: 0.8042
Epoch 12/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.5117 - acc: 0.8275 - val_loss: 0.5426 - val_acc: 0.7963
Epoch 13/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4844 - acc: 0.8336 - val_loss: 0.4514 - val_acc: 0.8350
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4482 - acc: 0.8507 - val_loss: 0.5428 - val_acc: 0.7772
Epoch 15/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4176 - acc: 0.8598 - val_loss: 0.4651 - val_acc: 0.8293
Epoch 16/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4042 - acc: 0.8643 - val_loss: 0.3482 - val_acc: 0.8999
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3795 - acc: 0.8720 - val_loss: 0.3890 - val_acc: 0.8628
Epoch 18/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3624 - acc: 0.8749 - val_loss: 0.3655 - val_acc: 0.8545
Epoch 19/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3509 - acc: 0.8811 - val_loss: 0.2883 - val_acc: 0.9187
Epoch 20/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3351 - acc: 0.8815 - val_loss: 0.3352 - val_acc: 0.8646
Epoch 21/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.3145 - acc: 0.8928 - val_loss: 0.3058 - val_acc: 0.8858
Epoch 22/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3222 - acc: 0.8855 - val_loss: 0.3160 - val_acc: 0.8921
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2961 - acc: 0.8966 - val_loss: 0.4609 - val_acc: 0.8050
Epoch 24/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2793 - acc: 0.9043 - val_loss: 0.4311 - val_acc: 0.8424
Epoch 25/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.2793 - acc: 0.9015 - val_loss: 0.2962 - val_acc: 0.8874
Epoch 26/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2939 - acc: 0.8943 - val_loss: 0.2560 - val_acc: 0.8966
Epoch 27/30
47401/47401 [==============================] - 5s 100us/step - loss: 0.2499 - acc: 0.9148 - val_loss: 0.2090 - val_acc: 0.9374
Epoch 28/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.2603 - acc: 0.9081 - val_loss: 0.2442 - val_acc: 0.9256
Epoch 29/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.2512 - acc: 0.9093 - val_loss: 0.1851 - val_acc: 0.9504
Epoch 30/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2305 - acc: 0.9219 - val_loss: 0.3569 - val_acc: 0.8563
iter 1500: eva_log=eva_acc=0.9219003818547715
iter 1510: g_log=[-1.4561727, 0.006297893, 1.7710674, 0.4375] d_log=[1.7969141, 0.3125]
iter 1520: g_log=[-1.4027085, 0.0066909874, 1.737258, 0.359375] d_log=[1.7584153, 0.328125]
iter 1530: g_log=[-1.4006906, 0.00604997, 1.703189, 0.25] d_log=[1.7739723, 0.140625]
iter 1540: g_log=[-1.3962528, 0.006183732, 1.7054393, 0.21875] d_log=[1.7430907, 0.203125]
iter 1550: g_log=[-1.4585719, 0.0049200365, 1.7045738, 0.515625] d_log=[1.695837, 0.34375]
iter 1560: g_log=[-1.5070767, 0.005724535, 1.7933035, 0.15625] d_log=[1.764885, 0.1875]
iter 1570: g_log=[-1.4349704, 0.005712903, 1.7206155, 0.25] d_log=[1.7246449, 0.265625]
iter 1580: g_log=[-1.4709733, 0.006327347, 1.7873406, 0.3125] d_log=[1.7974937, 0.3125]
iter 1590: g_log=[-1.4887431, 0.0054990184, 1.7636939, 0.21875] d_log=[1.7617646, 0.234375]
iter 1600: g_log=[-1.4869365, 0.0052874824, 1.7513106, 0.234375] d_log=[1.7272263, 0.21875]
iter 1610: g_log=[-1.4493219, 0.0058312775, 1.7408857, 0.203125] d_log=[1.8077149, 0.140625]
iter 1620: g_log=[-1.4083402, 0.0061624423, 1.7164624, 0.375] d_log=[1.7153802, 0.40625]
iter 1630: g_log=[-1.3870001, 0.007412144, 1.7576072, 0.40625] d_log=[1.7264891, 0.375]
iter 1640: g_log=[-1.4576954, 0.0059263897, 1.7540148, 0.140625] d_log=[1.7371624, 0.234375]
iter 1650: g_log=[-1.4657024, 0.00508106, 1.7197554, 0.46875] d_log=[1.7351863, 0.390625]
iter 1660: g_log=[-1.4379281, 0.0053750733, 1.7066817, 0.25] d_log=[1.7436976, 0.21875]
iter 1670: g_log=[-1.4988726, 0.005055516, 1.7516484, 0.140625] d_log=[1.7801269, 0.140625]
iter 1680: g_log=[-1.4921906, 0.0048653972, 1.7354604, 0.25] d_log=[1.7312397, 0.265625]
iter 1690: g_log=[-1.4928675, 0.0061003678, 1.7978859, 0.15625] d_log=[1.8227959, 0.140625]
iter 1700: g_log=[-1.5290184, 0.004865936, 1.7723153, 0.1875] d_log=[1.7470498, 0.234375]
iter 1710: g_log=[-1.4940678, 0.00506851, 1.7474933, 0.15625] d_log=[1.7397829, 0.265625]
iter 1720: g_log=[-1.5361032, 0.0047896756, 1.775587, 0.234375] d_log=[1.742271, 0.15625]
iter 1730: g_log=[-1.5056856, 0.0052298787, 1.7671795, 0.15625] d_log=[1.7749414, 0.15625]
iter 1740: g_log=[-1.4486698, 0.0062021944, 1.7587795, 0.21875] d_log=[1.772896, 0.203125]
iter 1750: g_log=[-1.5345241, 0.0051367446, 1.7913613, 0.1875] d_log=[1.7236543, 0.265625]
iter 1760: g_log=[-1.4830815, 0.0059932256, 1.7827427, 0.109375] d_log=[1.7015603, 0.28125]
iter 1770: g_log=[-1.5307548, 0.0051667737, 1.7890935, 0.203125] d_log=[1.7398802, 0.15625]
iter 1780: g_log=[-1.455254, 0.005630858, 1.7367969, 0.234375] d_log=[1.7495689, 0.171875]
iter 1790: g_log=[-1.4506398, 0.005936487, 1.7474642, 0.359375] d_log=[1.78528, 0.125]
iter 1800: g_log=[-1.4921418, 0.005351631, 1.7597234, 0.234375] d_log=[1.7619743, 0.203125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 198us/step - loss: 1.9624 - acc: 0.2415 - val_loss: 1.7636 - val_acc: 0.3235
Epoch 2/30
47401/47401 [==============================] - 5s 105us/step - loss: 1.5797 - acc: 0.3949 - val_loss: 1.5017 - val_acc: 0.4832
Epoch 3/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.3502 - acc: 0.4878 - val_loss: 1.2653 - val_acc: 0.4892
Epoch 4/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.1925 - acc: 0.5526 - val_loss: 1.1243 - val_acc: 0.5319
Epoch 5/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.0652 - acc: 0.6091 - val_loss: 0.9800 - val_acc: 0.6483
Epoch 6/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.9610 - acc: 0.6463 - val_loss: 0.9443 - val_acc: 0.6537
Epoch 7/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.8618 - acc: 0.6921 - val_loss: 0.8207 - val_acc: 0.7250
Epoch 8/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.7902 - acc: 0.7154 - val_loss: 0.8153 - val_acc: 0.7052
Epoch 9/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.7343 - acc: 0.7337 - val_loss: 0.6880 - val_acc: 0.7602
Epoch 10/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6704 - acc: 0.7590 - val_loss: 0.5738 - val_acc: 0.8301
Epoch 11/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.6109 - acc: 0.7832 - val_loss: 0.5506 - val_acc: 0.8219
Epoch 12/30
47401/47401 [==============================] - 5s 106us/step - loss: 0.6055 - acc: 0.7761 - val_loss: 0.6105 - val_acc: 0.7470
Epoch 13/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5548 - acc: 0.7984 - val_loss: 0.5119 - val_acc: 0.8275
Epoch 14/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.5162 - acc: 0.8129 - val_loss: 0.4681 - val_acc: 0.8475
Epoch 15/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.5101 - acc: 0.8144 - val_loss: 0.5022 - val_acc: 0.7935
Epoch 16/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4749 - acc: 0.8278 - val_loss: 0.4601 - val_acc: 0.8515
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4692 - acc: 0.8299 - val_loss: 0.7304 - val_acc: 0.7266
Epoch 18/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4527 - acc: 0.8348 - val_loss: 0.5294 - val_acc: 0.7870
Epoch 19/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4262 - acc: 0.8452 - val_loss: 0.3851 - val_acc: 0.8577
Epoch 20/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4243 - acc: 0.8455 - val_loss: 0.5016 - val_acc: 0.8241
Epoch 21/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4164 - acc: 0.8468 - val_loss: 0.3344 - val_acc: 0.8995
Epoch 22/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3849 - acc: 0.8597 - val_loss: 0.6781 - val_acc: 0.7417
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3718 - acc: 0.8635 - val_loss: 0.5689 - val_acc: 0.7585
Epoch 24/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3600 - acc: 0.8687 - val_loss: 0.3766 - val_acc: 0.8544
Epoch 25/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3637 - acc: 0.8654 - val_loss: 0.4050 - val_acc: 0.8279
Epoch 26/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3454 - acc: 0.8773 - val_loss: 0.2932 - val_acc: 0.9133
Epoch 27/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3251 - acc: 0.8848 - val_loss: 0.3306 - val_acc: 0.8913
Epoch 28/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3281 - acc: 0.8797 - val_loss: 0.4863 - val_acc: 0.8105
Epoch 29/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3247 - acc: 0.8817 - val_loss: 0.2834 - val_acc: 0.9117
Epoch 30/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.2932 - acc: 0.8961 - val_loss: 0.3811 - val_acc: 0.8500
iter 1800: eva_log=eva_acc=0.8961414316128519
iter 1810: g_log=[-1.4951173, 0.005505178, 1.7703762, 0.15625] d_log=[1.7936281, 0.125]
iter 1820: g_log=[-1.522642, 0.005091033, 1.7771937, 0.203125] d_log=[1.7707437, 0.21875]
iter 1830: g_log=[-1.5106726, 0.005466704, 1.7840078, 0.125] d_log=[1.7598864, 0.25]
iter 1840: g_log=[-1.517574, 0.0049291495, 1.7640314, 0.171875] d_log=[1.7469969, 0.21875]
iter 1850: g_log=[-1.5215036, 0.00545228, 1.7941176, 0.125] d_log=[1.7329917, 0.265625]
iter 1860: g_log=[-1.3636525, 0.0067056217, 1.6989336, 0.25] d_log=[1.7222058, 0.171875]
iter 1870: g_log=[-1.5001566, 0.00544867, 1.7725902, 0.265625] d_log=[1.7562888, 0.28125]
iter 1880: g_log=[-1.5194468, 0.004831282, 1.761011, 0.15625] d_log=[1.7601501, 0.171875]
iter 1890: g_log=[-1.5147953, 0.0050577642, 1.7676835, 0.21875] d_log=[1.7571027, 0.28125]
iter 1900: g_log=[-1.4798901, 0.0049704746, 1.7284138, 0.28125] d_log=[1.7397615, 0.171875]
iter 1910: g_log=[-1.3989525, 0.007203792, 1.7591422, 0.125] d_log=[1.7871108, 0.125]
iter 1920: g_log=[-1.4193254, 0.006408026, 1.7397267, 0.203125] d_log=[1.7436565, 0.15625]
iter 1930: g_log=[-1.511812, 0.00505956, 1.7647899, 0.171875] d_log=[1.7769969, 0.21875]
iter 1940: g_log=[-1.3371063, 0.0049227364, 1.5832431, 0.359375] d_log=[1.4111015, 0.65625]
iter 1950: g_log=[-1.5146608, 0.005467392, 1.7880305, 0.203125] d_log=[1.773319, 0.234375]
iter 1960: g_log=[-1.4481112, 0.0055793975, 1.7270811, 0.171875] d_log=[1.6836843, 0.34375]
iter 1970: g_log=[-1.4800712, 0.004983499, 1.7292461, 0.203125] d_log=[1.6667033, 0.1875]
iter 1980: g_log=[-1.4022882, 0.00679361, 1.7419686, 0.390625] d_log=[1.7536383, 0.34375]
iter 1990: g_log=[-1.4673624, 0.005330769, 1.7339008, 0.125] d_log=[1.7308758, 0.171875]
iter 2000: g_log=[-1.4445046, 0.0053400975, 1.7115095, 0.265625] d_log=[1.7413996, 0.203125]
iter 2010: g_log=[-1.5086434, 0.005702433, 1.7937651, 0.171875] d_log=[1.7349002, 0.21875]
iter 2020: g_log=[-1.5703723, 0.004758273, 1.808286, 0.140625] d_log=[1.8062625, 0.125]
iter 2030: g_log=[-1.5669982, 0.004346398, 1.7843181, 0.078125] d_log=[1.786685, 0.140625]
iter 2040: g_log=[-1.5145175, 0.0049528372, 1.7621593, 0.1875] d_log=[1.8157206, 0.15625]
iter 2050: g_log=[-1.5518428, 0.0046778377, 1.7857347, 0.140625] d_log=[1.7683582, 0.140625]
iter 2060: g_log=[-1.5280877, 0.00503793, 1.7799842, 0.15625] d_log=[1.7804929, 0.21875]
iter 2070: g_log=[-1.483296, 0.005406812, 1.7536366, 0.21875] d_log=[1.7407057, 0.1875]
iter 2080: g_log=[-1.3324593, 0.00696275, 1.6805968, 0.21875] d_log=[1.7598286, 0.15625]
iter 2090: g_log=[-1.4417593, 0.005184046, 1.7009616, 0.21875] d_log=[1.7098083, 0.25]
iter 2100: g_log=[-1.4451089, 0.006615662, 1.775892, 0.1875] d_log=[1.7298319, 0.125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 200us/step - loss: 6.0043 - acc: 0.2116 - val_loss: 5.7709 - val_acc: 0.3591
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.7682 - acc: 0.3303 - val_loss: 5.7205 - val_acc: 0.3397
Epoch 3/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.6074 - acc: 0.4112 - val_loss: 5.4987 - val_acc: 0.4444
Epoch 4/30
47401/47401 [==============================] - 5s 105us/step - loss: 5.4898 - acc: 0.4697 - val_loss: 5.3807 - val_acc: 0.5351
Epoch 5/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.4173 - acc: 0.4931 - val_loss: 5.3758 - val_acc: 0.4989
Epoch 6/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.3611 - acc: 0.5167 - val_loss: 5.2633 - val_acc: 0.5626
Epoch 7/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.3033 - acc: 0.5404 - val_loss: 5.2674 - val_acc: 0.5338
Epoch 8/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.2651 - acc: 0.5501 - val_loss: 5.1835 - val_acc: 0.5973
Epoch 9/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.2417 - acc: 0.5569 - val_loss: 5.2862 - val_acc: 0.5283
Epoch 10/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.2254 - acc: 0.5652 - val_loss: 5.2396 - val_acc: 0.5647
Epoch 11/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.1991 - acc: 0.5729 - val_loss: 5.1195 - val_acc: 0.6102
Epoch 12/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.0934 - acc: 0.5654 - val_loss: 0.9319 - val_acc: 0.6328
Epoch 13/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.8452 - acc: 0.6878 - val_loss: 0.7541 - val_acc: 0.7243
Epoch 14/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.7547 - acc: 0.7204 - val_loss: 0.7086 - val_acc: 0.7535
Epoch 15/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.6918 - acc: 0.7422 - val_loss: 0.6164 - val_acc: 0.7792
Epoch 16/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.6378 - acc: 0.7629 - val_loss: 0.5722 - val_acc: 0.8010
Epoch 17/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.6137 - acc: 0.7702 - val_loss: 0.5733 - val_acc: 0.7774
Epoch 18/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.5695 - acc: 0.7875 - val_loss: 0.5827 - val_acc: 0.7968
Epoch 19/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.5542 - acc: 0.7906 - val_loss: 0.5362 - val_acc: 0.7971
Epoch 20/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.5189 - acc: 0.8076 - val_loss: 0.5125 - val_acc: 0.8024
Epoch 21/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4913 - acc: 0.8204 - val_loss: 0.4869 - val_acc: 0.8227
Epoch 22/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4867 - acc: 0.8180 - val_loss: 0.4493 - val_acc: 0.8470
Epoch 23/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4634 - acc: 0.8239 - val_loss: 0.4937 - val_acc: 0.7965
Epoch 24/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4374 - acc: 0.8369 - val_loss: 0.3956 - val_acc: 0.8671
Epoch 25/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4172 - acc: 0.8447 - val_loss: 0.5851 - val_acc: 0.7816
Epoch 26/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.4149 - acc: 0.8425 - val_loss: 0.4157 - val_acc: 0.8588
Epoch 27/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.4012 - acc: 0.8509 - val_loss: 0.3527 - val_acc: 0.8833
Epoch 28/30
47401/47401 [==============================] - 5s 102us/step - loss: 0.3980 - acc: 0.8500 - val_loss: 0.5291 - val_acc: 0.7516
Epoch 29/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3774 - acc: 0.8595 - val_loss: 0.4060 - val_acc: 0.8379
Epoch 30/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3606 - acc: 0.8673 - val_loss: 0.4985 - val_acc: 0.8011
iter 2100: eva_log=eva_acc=0.8672812809718012
iter 2110: g_log=[-1.5189389, 0.004754451, 1.7566614, 0.203125] d_log=[1.7270815, 0.203125]
iter 2120: g_log=[-1.5501508, 0.00497103, 1.7987022, 0.140625] d_log=[1.7891762, 0.171875]
iter 2130: g_log=[-1.5382904, 0.004794776, 1.7780292, 0.1875] d_log=[1.75309, 0.15625]
iter 2140: g_log=[-1.4753219, 0.006322729, 1.7914584, 0.109375] d_log=[1.7659562, 0.140625]
iter 2150: g_log=[-1.5089617, 0.005231791, 1.7705512, 0.15625] d_log=[1.7320545, 0.25]
iter 2160: g_log=[-1.5472132, 0.004941427, 1.7942846, 0.171875] d_log=[1.743255, 0.1875]
iter 2170: g_log=[-1.5411742, 0.0046427227, 1.7733103, 0.203125] d_log=[1.7434174, 0.28125]
iter 2180: g_log=[-1.4471841, 0.00559828, 1.7270981, 0.203125] d_log=[1.7671757, 0.171875]
iter 2190: g_log=[-1.5600368, 0.004699623, 1.795018, 0.328125] d_log=[1.7840583, 0.3125]
iter 2200: g_log=[-1.4958647, 0.004693724, 1.730551, 0.46875] d_log=[1.7351787, 0.4375]
iter 2210: g_log=[-1.5643773, 0.0043095844, 1.7798566, 0.15625] d_log=[1.729412, 0.234375]
iter 2220: g_log=[-1.5572613, 0.004451753, 1.779849, 0.203125] d_log=[1.7523552, 0.21875]
iter 2230: g_log=[-1.5609391, 0.0043078866, 1.7763335, 0.21875] d_log=[1.7633686, 0.3125]
iter 2240: g_log=[-1.574981, 0.004269396, 1.7884507, 0.1875] d_log=[1.7871534, 0.1875]
iter 2250: g_log=[-1.5445461, 0.0044301082, 1.7660515, 0.203125] d_log=[1.7941086, 0.171875]
iter 2260: g_log=[-1.5360467, 0.004286171, 1.7503552, 0.40625] d_log=[1.7514307, 0.265625]
iter 2270: g_log=[-1.4476479, 0.005128376, 1.7040668, 0.265625] d_log=[1.7137022, 0.265625]
iter 2280: g_log=[-1.4920684, 0.0046319566, 1.7236662, 0.40625] d_log=[1.7567413, 0.3125]
iter 2290: g_log=[-1.4996536, 0.004724238, 1.7358655, 0.203125] d_log=[1.740339, 0.140625]
iter 2300: g_log=[-1.5571053, 0.0044708196, 1.7806463, 0.15625] d_log=[1.7413732, 0.203125]
iter 2310: g_log=[-1.5303144, 0.0056580696, 1.8132179, 0.171875] d_log=[1.755635, 0.15625]
iter 2320: g_log=[-1.5757011, 0.004648326, 1.8081174, 0.15625] d_log=[1.7737064, 0.3125]
iter 2330: g_log=[-1.5200408, 0.0051877913, 1.7794304, 0.171875] d_log=[1.7854295, 0.265625]
iter 2340: g_log=[-1.5596123, 0.005141873, 1.816706, 0.171875] d_log=[1.8080156, 0.0]
iter 2350: g_log=[-1.5536857, 0.004782872, 1.7928293, 0.234375] d_log=[1.798835, 0.203125]
iter 2360: g_log=[-1.5512823, 0.004439654, 1.773265, 0.15625] d_log=[1.7025998, 0.265625]
iter 2370: g_log=[-1.510326, 0.0045888275, 1.7397674, 0.328125] d_log=[1.7356136, 0.40625]
iter 2380: g_log=[-1.4775684, 0.0051601613, 1.7355764, 0.171875] d_log=[1.76888, 0.140625]
iter 2390: g_log=[-1.514373, 0.0040324004, 1.7159929, 0.234375] d_log=[1.7738675, 0.15625]
iter 2400: g_log=[-1.485994, 0.005285132, 1.7502506, 0.15625] d_log=[1.7628974, 0.15625]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 199us/step - loss: 4.0599 - acc: 0.2396 - val_loss: 4.0187 - val_acc: 0.1464
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 3.7255 - acc: 0.3717 - val_loss: 3.5824 - val_acc: 0.4086
Epoch 3/30
47401/47401 [==============================] - 5s 105us/step - loss: 3.5392 - acc: 0.4531 - val_loss: 3.4535 - val_acc: 0.4958
Epoch 4/30
47401/47401 [==============================] - 5s 103us/step - loss: 3.4173 - acc: 0.5002 - val_loss: 3.3418 - val_acc: 0.5262
Epoch 5/30
47401/47401 [==============================] - 5s 104us/step - loss: 3.3321 - acc: 0.5362 - val_loss: 3.2445 - val_acc: 0.5868
Epoch 6/30
47401/47401 [==============================] - 5s 103us/step - loss: 3.2626 - acc: 0.5600 - val_loss: 3.2770 - val_acc: 0.5478
Epoch 7/30
47401/47401 [==============================] - 5s 102us/step - loss: 3.1954 - acc: 0.5918 - val_loss: 3.2088 - val_acc: 0.5888
Epoch 8/30
47401/47401 [==============================] - 5s 105us/step - loss: 3.1323 - acc: 0.6178 - val_loss: 3.1413 - val_acc: 0.5345
Epoch 9/30
47401/47401 [==============================] - 5s 103us/step - loss: 3.0965 - acc: 0.6294 - val_loss: 3.0101 - val_acc: 0.6628
Epoch 10/30
47401/47401 [==============================] - 5s 103us/step - loss: 3.0456 - acc: 0.6529 - val_loss: 3.0387 - val_acc: 0.6495
Epoch 11/30
47401/47401 [==============================] - 5s 104us/step - loss: 3.0104 - acc: 0.6633 - val_loss: 3.0051 - val_acc: 0.6526
Epoch 12/30
47401/47401 [==============================] - 5s 105us/step - loss: 2.9815 - acc: 0.6734 - val_loss: 2.9272 - val_acc: 0.6830
Epoch 13/30
47401/47401 [==============================] - 5s 105us/step - loss: 2.9308 - acc: 0.6946 - val_loss: 2.9186 - val_acc: 0.6594
Epoch 14/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.8963 - acc: 0.7111 - val_loss: 2.8521 - val_acc: 0.7107
Epoch 15/30
47401/47401 [==============================] - 5s 105us/step - loss: 2.8871 - acc: 0.7096 - val_loss: 2.8649 - val_acc: 0.7032
Epoch 16/30
47401/47401 [==============================] - 5s 105us/step - loss: 2.8601 - acc: 0.7218 - val_loss: 3.0098 - val_acc: 0.6507
Epoch 17/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.8440 - acc: 0.7266 - val_loss: 2.8943 - val_acc: 0.6809
Epoch 18/30
47401/47401 [==============================] - 5s 102us/step - loss: 2.8332 - acc: 0.7277 - val_loss: 2.7746 - val_acc: 0.7473
Epoch 19/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.8120 - acc: 0.7396 - val_loss: 2.8896 - val_acc: 0.6953
Epoch 20/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.8056 - acc: 0.7382 - val_loss: 2.7388 - val_acc: 0.7588
Epoch 21/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.7809 - acc: 0.7504 - val_loss: 2.8015 - val_acc: 0.7392
Epoch 22/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.7786 - acc: 0.7491 - val_loss: 2.7299 - val_acc: 0.7513
Epoch 23/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.7678 - acc: 0.7530 - val_loss: 2.7262 - val_acc: 0.7622
Epoch 24/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.7501 - acc: 0.7604 - val_loss: 2.7299 - val_acc: 0.7721
Epoch 25/30
47401/47401 [==============================] - 5s 105us/step - loss: 2.7467 - acc: 0.7611 - val_loss: 2.7028 - val_acc: 0.7675
Epoch 26/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.7372 - acc: 0.7664 - val_loss: 2.6680 - val_acc: 0.7978
Epoch 27/30
47401/47401 [==============================] - 5s 103us/step - loss: 2.7445 - acc: 0.7619 - val_loss: 2.7159 - val_acc: 0.7712
Epoch 28/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.7259 - acc: 0.7703 - val_loss: 2.6739 - val_acc: 0.7956
Epoch 29/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.7239 - acc: 0.7679 - val_loss: 2.7636 - val_acc: 0.7363
Epoch 30/30
47401/47401 [==============================] - 5s 104us/step - loss: 2.7078 - acc: 0.7767 - val_loss: 2.8062 - val_acc: 0.7295
iter 2400: eva_log=eva_acc=0.7766924748485902
iter 2410: g_log=[-1.4602113, 0.0051557766, 1.7180002, 0.328125] d_log=[1.7117314, 0.453125]
iter 2420: g_log=[-1.5335457, 0.0047618896, 1.7716402, 0.203125] d_log=[1.742484, 0.234375]
iter 2430: g_log=[-1.5635362, 0.0043116533, 1.7791189, 0.234375] d_log=[1.7986073, 0.1875]
iter 2440: g_log=[-1.5724435, 0.004528689, 1.798878, 0.140625] d_log=[1.7828805, 0.265625]
iter 2450: g_log=[-1.2435588, 0.0052377037, 1.5054439, 0.421875] d_log=[1.4779129, 0.453125]
iter 2460: g_log=[-1.3762106, 0.0048880465, 1.6206129, 0.359375] d_log=[1.6445353, 0.21875]
iter 2470: g_log=[-1.4752611, 0.004959699, 1.7232461, 0.25] d_log=[1.7529335, 0.4375]
iter 2480: g_log=[-1.4941272, 0.005729448, 1.7805996, 0.375] d_log=[1.7239482, 0.484375]
iter 2490: g_log=[-1.5525628, 0.00415252, 1.7601888, 0.125] d_log=[1.76247, 0.0625]
iter 2500: g_log=[-1.4817798, 0.0063608014, 1.79982, 0.125] d_log=[1.7608731, 0.140625]
iter 2510: g_log=[-1.4973111, 0.004800683, 1.7373452, 0.3125] d_log=[1.7439945, 0.328125]
iter 2520: g_log=[-1.4848821, 0.005158053, 1.7427847, 0.40625] d_log=[1.7870972, 0.234375]
iter 2530: g_log=[-1.5427125, 0.0046366462, 1.7745447, 0.203125] d_log=[1.6948695, 0.296875]
iter 2540: g_log=[-1.5561703, 0.003928445, 1.7525926, 0.15625] d_log=[1.7391376, 0.25]
iter 2550: g_log=[-1.4996109, 0.0047876723, 1.7389945, 0.203125] d_log=[1.7564166, 0.21875]
iter 2560: g_log=[-1.5079356, 0.004922798, 1.7540755, 0.0] d_log=[1.783375, 0.0]
iter 2570: g_log=[-1.5038687, 0.005366218, 1.7721796, 0.21875] d_log=[1.7824831, 0.171875]
iter 2580: g_log=[-1.5362958, 0.0040787626, 1.7402339, 0.453125] d_log=[1.76161, 0.328125]
iter 2590: g_log=[-1.4913812, 0.004879046, 1.7353334, 0.21875] d_log=[1.7607, 0.203125]
iter 2600: g_log=[-1.5166149, 0.00474568, 1.7538989, 0.171875] d_log=[1.7420301, 0.171875]
iter 2610: g_log=[-1.4584807, 0.0055150436, 1.7342329, 0.15625] d_log=[1.7670629, 0.171875]
iter 2620: g_log=[-1.2072387, 0.009464561, 1.6804667, 0.265625] d_log=[1.7994967, 0.140625]
iter 2630: g_log=[-1.4405686, 0.0053553586, 1.7083365, 0.265625] d_log=[1.787768, 0.171875]
iter 2640: g_log=[-1.5414532, 0.0047070533, 1.7768059, 0.375] d_log=[1.7650707, 0.40625]
iter 2650: g_log=[-1.5522255, 0.00431847, 1.768149, 0.234375] d_log=[1.7741681, 0.234375]
iter 2660: g_log=[-1.5567296, 0.004953779, 1.8044184, 0.21875] d_log=[1.8457916, 0.140625]
iter 2670: g_log=[-1.5625483, 0.0044667497, 1.7858858, 0.1875] d_log=[1.7853726, 0.21875]
iter 2680: g_log=[-1.5356873, 0.0045300266, 1.7621887, 0.265625] d_log=[1.7710323, 0.21875]
iter 2690: g_log=[-1.529851, 0.00465402, 1.762552, 0.265625] d_log=[1.7832277, 0.25]
iter 2700: g_log=[-1.557739, 0.0047225943, 1.7938688, 0.203125] d_log=[1.7201381, 0.265625]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 9s 198us/step - loss: 1.9856 - acc: 0.2152 - val_loss: 1.7671 - val_acc: 0.2623
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.6701 - acc: 0.3611 - val_loss: 1.5533 - val_acc: 0.4154
Epoch 3/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.4290 - acc: 0.4808 - val_loss: 1.4315 - val_acc: 0.4059
Epoch 4/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.2015 - acc: 0.5782 - val_loss: 1.0636 - val_acc: 0.6714
Epoch 5/30
47401/47401 [==============================] - 5s 106us/step - loss: 1.0532 - acc: 0.6281 - val_loss: 0.9351 - val_acc: 0.6956
Epoch 6/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.9254 - acc: 0.6787 - val_loss: 0.9997 - val_acc: 0.6141
Epoch 7/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.8198 - acc: 0.7167 - val_loss: 0.7591 - val_acc: 0.7305
Epoch 8/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.7465 - acc: 0.7411 - val_loss: 0.7450 - val_acc: 0.7583
Epoch 9/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.6870 - acc: 0.7606 - val_loss: 0.6116 - val_acc: 0.7994
Epoch 10/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.6206 - acc: 0.7852 - val_loss: 1.0151 - val_acc: 0.5851
Epoch 11/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.5948 - acc: 0.7919 - val_loss: 0.5545 - val_acc: 0.8157
Epoch 12/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.5523 - acc: 0.8062 - val_loss: 0.5128 - val_acc: 0.8526
Epoch 13/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.5187 - acc: 0.8196 - val_loss: 0.7026 - val_acc: 0.6634
Epoch 14/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.4979 - acc: 0.8222 - val_loss: 0.4433 - val_acc: 0.8455
Epoch 15/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4791 - acc: 0.8271 - val_loss: 0.7939 - val_acc: 0.6481
Epoch 16/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.4505 - acc: 0.8415 - val_loss: 0.4715 - val_acc: 0.8239
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4297 - acc: 0.8465 - val_loss: 0.4041 - val_acc: 0.8690
Epoch 18/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.4164 - acc: 0.8504 - val_loss: 0.3869 - val_acc: 0.8586
Epoch 19/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.3953 - acc: 0.8564 - val_loss: 0.3819 - val_acc: 0.8588
Epoch 20/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3771 - acc: 0.8661 - val_loss: 0.4089 - val_acc: 0.8412
Epoch 21/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3651 - acc: 0.8671 - val_loss: 0.3396 - val_acc: 0.8702
Epoch 22/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3666 - acc: 0.8634 - val_loss: 0.3042 - val_acc: 0.9027
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3628 - acc: 0.8678 - val_loss: 0.5097 - val_acc: 0.8001
Epoch 24/30
47401/47401 [==============================] - 5s 106us/step - loss: 0.3422 - acc: 0.8765 - val_loss: 0.2974 - val_acc: 0.8918
Epoch 25/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.3430 - acc: 0.8767 - val_loss: 0.2535 - val_acc: 0.9211
Epoch 26/30
47401/47401 [==============================] - 5s 106us/step - loss: 0.3255 - acc: 0.8838 - val_loss: 0.2967 - val_acc: 0.9048
Epoch 27/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3296 - acc: 0.8826 - val_loss: 0.3261 - val_acc: 0.8860
Epoch 28/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2968 - acc: 0.8954 - val_loss: 0.3223 - val_acc: 0.8793
Epoch 29/30
47401/47401 [==============================] - 5s 103us/step - loss: 0.3057 - acc: 0.8904 - val_loss: 0.2918 - val_acc: 0.8874
Epoch 30/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.2898 - acc: 0.8966 - val_loss: 0.2698 - val_acc: 0.9125
iter 2700: eva_log=eva_acc=0.8965844602635741
iter 2710: g_log=[-1.5073538, 0.0046698702, 1.7408473, 0.1875] d_log=[1.7808994, 0.171875]
iter 2720: g_log=[-1.4667565, 0.0054064635, 1.7370796, 0.234375] d_log=[1.7547619, 0.1875]
iter 2730: g_log=[-1.54729, 0.0050091674, 1.7977483, 0.171875] d_log=[1.7125759, 0.28125]
iter 2740: g_log=[-1.5034239, 0.004445374, 1.7256926, 0.21875] d_log=[1.7137663, 0.203125]
iter 2750: g_log=[-1.4736886, 0.005113862, 1.7293817, 0.265625] d_log=[1.7238138, 0.265625]
iter 2760: g_log=[-1.564708, 0.0043460466, 1.7820103, 0.328125] d_log=[1.7293097, 0.46875]
iter 2770: g_log=[-1.562818, 0.0037141312, 1.7485247, 0.375] d_log=[1.7568967, 0.171875]
iter 2780: g_log=[-1.5267518, 0.0048506698, 1.7692852, 0.171875] d_log=[1.7686839, 0.1875]
iter 2790: g_log=[-1.5383003, 0.0050021745, 1.788409, 0.140625] d_log=[1.7637132, 0.1875]
iter 2800: g_log=[-1.5686202, 0.0048641097, 1.8118258, 0.171875] d_log=[1.7392387, 0.1875]
iter 2810: g_log=[-1.5391113, 0.0047297217, 1.7755973, 0.390625] d_log=[1.7122118, 0.515625]
iter 2820: g_log=[-1.5108984, 0.0043143835, 1.7266176, 0.28125] d_log=[1.753669, 0.203125]
iter 2830: g_log=[-1.4917064, 0.0051577766, 1.7495952, 0.265625] d_log=[1.7302661, 0.28125]
iter 2840: g_log=[-1.5573921, 0.004201919, 1.767488, 0.15625] d_log=[1.7569151, 0.1875]
iter 2850: g_log=[-1.5119536, 0.004406574, 1.7322823, 0.453125] d_log=[1.7640488, 0.375]
iter 2860: g_log=[-1.5643642, 0.0039679557, 1.762762, 0.203125] d_log=[1.7566774, 0.234375]
iter 2870: g_log=[-1.5653477, 0.0049157124, 1.8111333, 0.125] d_log=[1.7448397, 0.25]
iter 2880: g_log=[-1.4633654, 0.005800812, 1.753406, 0.171875] d_log=[1.7523745, 0.25]
iter 2890: g_log=[-1.581241, 0.0046682637, 1.8146542, 0.171875] d_log=[1.7584498, 0.25]
iter 2900: g_log=[-1.5337346, 0.0040976154, 1.7386153, 0.234375] d_log=[1.719115, 0.25]
iter 2910: g_log=[-1.5244093, 0.004613489, 1.7550838, 0.265625] d_log=[1.7216516, 0.328125]
iter 2920: g_log=[-1.4880565, 0.00540046, 1.7580795, 0.25] d_log=[1.730784, 0.28125]
iter 2930: g_log=[-1.550621, 0.004601444, 1.7806933, 0.328125] d_log=[1.7378199, 0.265625]
iter 2940: g_log=[-1.5240645, 0.004561593, 1.7521442, 0.171875] d_log=[1.7761021, 0.140625]
iter 2950: g_log=[-1.4403777, 0.0059843864, 1.739597, 0.1875] d_log=[1.7630346, 0.15625]
iter 2960: g_log=[-1.5170052, 0.0047471104, 1.7543607, 0.21875] d_log=[1.7197101, 0.1875]
iter 2970: g_log=[-1.4989853, 0.0049912697, 1.7485487, 0.40625] d_log=[1.751982, 0.203125]
iter 2980: g_log=[-1.5160247, 0.0044320086, 1.7376251, 0.15625] d_log=[1.7892768, 0.25]
iter 2990: g_log=[-1.5612136, 0.004465974, 1.7845123, 0.171875] d_log=[1.7787852, 0.109375]
iter 3000: g_log=[-1.5003685, 0.0046229986, 1.7315184, 0.28125] d_log=[1.7475314, 0.53125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 201us/step - loss: 11.6860 - acc: 0.2016 - val_loss: 11.6384 - val_acc: 0.1788
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 11.6082 - acc: 0.2399 - val_loss: 11.5374 - val_acc: 0.2618
Epoch 3/30
47401/47401 [==============================] - 5s 103us/step - loss: 11.5888 - acc: 0.2498 - val_loss: 11.5311 - val_acc: 0.2565
Epoch 4/30
47401/47401 [==============================] - 5s 104us/step - loss: 11.5714 - acc: 0.2558 - val_loss: 11.5178 - val_acc: 0.2696
Epoch 5/30
47401/47401 [==============================] - 5s 104us/step - loss: 11.5551 - acc: 0.2652 - val_loss: 11.5080 - val_acc: 0.2712
Epoch 6/30
47401/47401 [==============================] - 5s 104us/step - loss: 11.5498 - acc: 0.2672 - val_loss: 11.5025 - val_acc: 0.2753
Epoch 7/30
47401/47401 [==============================] - 5s 105us/step - loss: 11.5418 - acc: 0.2714 - val_loss: 11.4909 - val_acc: 0.2834
Epoch 8/30
47401/47401 [==============================] - 5s 104us/step - loss: 11.5385 - acc: 0.2718 - val_loss: 11.5128 - val_acc: 0.2693
Epoch 9/30
47401/47401 [==============================] - 5s 105us/step - loss: 11.5332 - acc: 0.2739 - val_loss: 11.4845 - val_acc: 0.2849
Epoch 10/30
47401/47401 [==============================] - 5s 103us/step - loss: 11.5296 - acc: 0.2760 - val_loss: 11.5236 - val_acc: 0.2583
Epoch 11/30
47401/47401 [==============================] - 5s 104us/step - loss: 9.9132 - acc: 0.2622 - val_loss: 1.7881 - val_acc: 0.2476
Epoch 12/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.6130 - acc: 0.3527 - val_loss: 1.4935 - val_acc: 0.4049
Epoch 13/30
47401/47401 [==============================] - 5s 105us/step - loss: 1.4525 - acc: 0.4153 - val_loss: 1.4364 - val_acc: 0.4192
Epoch 14/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.3865 - acc: 0.4395 - val_loss: 1.3494 - val_acc: 0.4635
Epoch 15/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.3482 - acc: 0.4426 - val_loss: 1.3142 - val_acc: 0.4575
Epoch 16/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.3048 - acc: 0.4480 - val_loss: 1.3445 - val_acc: 0.3983
Epoch 17/30
47401/47401 [==============================] - 5s 105us/step - loss: 1.2716 - acc: 0.4536 - val_loss: 1.3341 - val_acc: 0.4177
Epoch 18/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.2435 - acc: 0.4612 - val_loss: 1.2142 - val_acc: 0.4984
Epoch 19/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.2132 - acc: 0.4651 - val_loss: 1.1928 - val_acc: 0.4918
Epoch 20/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.1861 - acc: 0.4730 - val_loss: 1.1553 - val_acc: 0.4854
Epoch 21/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.1613 - acc: 0.4830 - val_loss: 1.1548 - val_acc: 0.4816
Epoch 22/30
47401/47401 [==============================] - 5s 106us/step - loss: 1.1478 - acc: 0.4919 - val_loss: 1.1475 - val_acc: 0.4886
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.1207 - acc: 0.5048 - val_loss: 1.1062 - val_acc: 0.5383
Epoch 24/30
47401/47401 [==============================] - 5s 102us/step - loss: 1.1080 - acc: 0.5131 - val_loss: 1.0979 - val_acc: 0.5158
Epoch 25/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.0908 - acc: 0.5231 - val_loss: 1.0629 - val_acc: 0.5376
Epoch 26/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.0731 - acc: 0.5299 - val_loss: 1.0905 - val_acc: 0.5383
Epoch 27/30
47401/47401 [==============================] - 5s 105us/step - loss: 1.0676 - acc: 0.5329 - val_loss: 1.2145 - val_acc: 0.4733
Epoch 28/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.0546 - acc: 0.5411 - val_loss: 1.0284 - val_acc: 0.5561
Epoch 29/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.0380 - acc: 0.5483 - val_loss: 1.0218 - val_acc: 0.5580
Epoch 30/30
47401/47401 [==============================] - 5s 106us/step - loss: 1.0326 - acc: 0.5509 - val_loss: 1.0378 - val_acc: 0.5605
iter 3000: eva_log=eva_acc=0.5509377439420275
iter 3010: g_log=[-1.4986062, 0.0051313033, 1.7551713, 0.1875] d_log=[1.7815804, 0.234375]
iter 3020: g_log=[-1.5509135, 0.0046034018, 1.7810836, 0.21875] d_log=[1.75777, 0.1875]
iter 3030: g_log=[-1.562228, 0.004694137, 1.7969348, 0.171875] d_log=[1.7767344, 0.140625]
iter 3040: g_log=[-1.3651171, 0.0062415646, 1.6771953, 0.390625] d_log=[1.7519767, 0.234375]
iter 3050: g_log=[-1.4506807, 0.005689951, 1.7351784, 0.296875] d_log=[1.7248242, 0.359375]
iter 3060: g_log=[-1.5313979, 0.0045308843, 1.7579422, 0.171875] d_log=[1.7735243, 0.09375]
iter 3070: g_log=[-1.5091673, 0.004385297, 1.7284322, 0.125] d_log=[1.7622904, 0.125]
iter 3080: g_log=[-1.5207542, 0.0048657684, 1.7640426, 0.34375] d_log=[1.7117963, 0.5]
iter 3090: g_log=[-1.5140679, 0.004543486, 1.7412422, 0.21875] d_log=[1.7296131, 0.15625]
iter 3100: g_log=[-1.5151368, 0.0046500135, 1.7476375, 0.265625] d_log=[1.7318659, 0.265625]
iter 3110: g_log=[-1.4912469, 0.005023781, 1.7424359, 0.25] d_log=[1.7591164, 0.203125]
iter 3120: g_log=[-1.5439075, 0.004649334, 1.7763742, 0.1875] d_log=[1.7813607, 0.171875]
iter 3130: g_log=[-1.5092435, 0.0051631588, 1.7674015, 0.234375] d_log=[1.7906071, 0.171875]
iter 3140: g_log=[-1.5337327, 0.0046227016, 1.7648677, 0.28125] d_log=[1.7752988, 0.234375]
iter 3150: g_log=[-1.463335, 0.0056772027, 1.7471951, 0.1875] d_log=[1.7952015, 0.265625]
iter 3160: g_log=[-1.4887028, 0.0046326164, 1.7203336, 0.5] d_log=[1.7847347, 0.359375]
iter 3170: g_log=[-1.5110441, 0.00513596, 1.7678422, 0.375] d_log=[1.7484422, 0.453125]
iter 3180: g_log=[-1.4952558, 0.004932362, 1.741874, 0.296875] d_log=[1.7461265, 0.21875]
iter 3190: g_log=[-1.5259221, 0.00459103, 1.7554736, 0.125] d_log=[1.7360278, 0.15625]
iter 3200: g_log=[-1.5219749, 0.0042868694, 1.7363183, 0.234375] d_log=[1.7463244, 0.234375]
iter 3210: g_log=[-1.0930667, 0.00477024, 1.3315787, 0.640625] d_log=[1.3754873, 0.578125]
iter 3220: g_log=[-1.5830369, 0.008014841, 1.983779, 0.34375] d_log=[2.0231395, 0.1875]
iter 3230: g_log=[-1.3355098, 0.0075156842, 1.7112939, 0.3125] d_log=[1.7361457, 0.234375]
iter 3240: g_log=[-1.45572, 0.0061289873, 1.7621694, 0.078125] d_log=[1.6868922, 0.21875]
iter 3250: g_log=[-1.4720399, 0.0051588966, 1.7299848, 0.25] d_log=[1.7277315, 0.28125]
iter 3260: g_log=[-1.2041881, 0.0049028816, 1.4493322, 0.421875] d_log=[1.4351696, 0.546875]
iter 3270: g_log=[-1.418141, 0.0072753755, 1.7819098, 0.015625] d_log=[1.7745447, 0.234375]
iter 3280: g_log=[-1.2710164, 0.009931896, 1.7676111, 0.171875] d_log=[1.7023315, 0.203125]
iter 3290: g_log=[-1.3390889, 0.0071984623, 1.699012, 0.25] d_log=[1.7151344, 0.203125]
iter 3300: g_log=[-1.3346688, 0.007299158, 1.6996267, 0.1875] d_log=[1.7136444, 0.203125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 202us/step - loss: 5.9114 - acc: 0.1908 - val_loss: 5.6923 - val_acc: 0.2582
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.6309 - acc: 0.2983 - val_loss: 5.4764 - val_acc: 0.3962
Epoch 3/30
47401/47401 [==============================] - 5s 105us/step - loss: 5.4859 - acc: 0.3662 - val_loss: 5.5201 - val_acc: 0.3359
Epoch 4/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.3985 - acc: 0.4004 - val_loss: 5.3368 - val_acc: 0.4201
Epoch 5/30
47401/47401 [==============================] - 5s 105us/step - loss: 5.3182 - acc: 0.4368 - val_loss: 5.3208 - val_acc: 0.4631
Epoch 6/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.2510 - acc: 0.4606 - val_loss: 5.1890 - val_acc: 0.5295
Epoch 7/30
47401/47401 [==============================] - 5s 105us/step - loss: 5.1946 - acc: 0.4897 - val_loss: 5.2720 - val_acc: 0.4086
Epoch 8/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.1561 - acc: 0.5009 - val_loss: 5.1336 - val_acc: 0.5586
Epoch 9/30
47401/47401 [==============================] - 5s 104us/step - loss: 5.0958 - acc: 0.5290 - val_loss: 5.0370 - val_acc: 0.5780
Epoch 10/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.0806 - acc: 0.5292 - val_loss: 5.1637 - val_acc: 0.4520
Epoch 11/30
47401/47401 [==============================] - 5s 102us/step - loss: 5.0466 - acc: 0.5463 - val_loss: 5.0300 - val_acc: 0.5843
Epoch 12/30
47401/47401 [==============================] - 5s 105us/step - loss: 5.0019 - acc: 0.5641 - val_loss: 5.0822 - val_acc: 0.5188
Epoch 13/30
47401/47401 [==============================] - 5s 103us/step - loss: 4.9940 - acc: 0.5636 - val_loss: 4.9749 - val_acc: 0.5857
Epoch 14/30
47401/47401 [==============================] - 5s 103us/step - loss: 4.9519 - acc: 0.5878 - val_loss: 5.0026 - val_acc: 0.5650
Epoch 15/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.9532 - acc: 0.5839 - val_loss: 4.9648 - val_acc: 0.5787
Epoch 16/30
47401/47401 [==============================] - 5s 102us/step - loss: 4.9322 - acc: 0.5926 - val_loss: 4.9408 - val_acc: 0.6137
Epoch 17/30
47401/47401 [==============================] - 5s 104us/step - loss: 4.9208 - acc: 0.5964 - val_loss: 4.9127 - val_acc: 0.5944
Epoch 18/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.8998 - acc: 0.6076 - val_loss: 4.8800 - val_acc: 0.6294
Epoch 19/30
47401/47401 [==============================] - 5s 103us/step - loss: 4.9059 - acc: 0.6014 - val_loss: 4.8596 - val_acc: 0.6326
Epoch 20/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.8715 - acc: 0.6211 - val_loss: 4.9353 - val_acc: 0.5931
Epoch 21/30
47401/47401 [==============================] - 5s 104us/step - loss: 4.8570 - acc: 0.6248 - val_loss: 4.8533 - val_acc: 0.6504
Epoch 22/30
47401/47401 [==============================] - 5s 104us/step - loss: 4.8539 - acc: 0.6259 - val_loss: 4.8263 - val_acc: 0.6575
Epoch 23/30
47401/47401 [==============================] - 5s 104us/step - loss: 4.8731 - acc: 0.6197 - val_loss: 4.8997 - val_acc: 0.6154
Epoch 24/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.8748 - acc: 0.6192 - val_loss: 4.8130 - val_acc: 0.6752
Epoch 25/30
47401/47401 [==============================] - 5s 103us/step - loss: 4.8420 - acc: 0.6313 - val_loss: 4.8917 - val_acc: 0.6135
Epoch 26/30
47401/47401 [==============================] - 5s 104us/step - loss: 4.8198 - acc: 0.6413 - val_loss: 4.8138 - val_acc: 0.6685
Epoch 27/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.8408 - acc: 0.6338 - val_loss: 4.8169 - val_acc: 0.6361
Epoch 28/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.8800 - acc: 0.6189 - val_loss: 4.7891 - val_acc: 0.6868
Epoch 29/30
47401/47401 [==============================] - 5s 104us/step - loss: 4.7967 - acc: 0.6558 - val_loss: 4.7818 - val_acc: 0.6781
Epoch 30/30
47401/47401 [==============================] - 5s 105us/step - loss: 4.8241 - acc: 0.6430 - val_loss: 4.7898 - val_acc: 0.6646
iter 3300: eva_log=eva_acc=0.6557878525719201
iter 3310: g_log=[-1.3876501, 0.0065763346, 1.7164669, 0.234375] d_log=[1.817152, 0.125]
iter 3320: g_log=[-1.4696406, 0.0060284194, 1.7710615, 0.15625] d_log=[1.7766443, 0.1875]
iter 3330: g_log=[-1.4737457, 0.0061528254, 1.781387, 0.203125] d_log=[1.7279159, 0.21875]
iter 3340: g_log=[-1.4950681, 0.005228076, 1.7564719, 0.203125] d_log=[1.7760626, 0.125]
iter 3350: g_log=[-1.5137388, 0.005292646, 1.7783711, 0.15625] d_log=[1.8081002, 0.140625]
iter 3360: g_log=[-1.2302399, 0.0096735535, 1.7139176, 0.203125] d_log=[1.7071649, 0.203125]
iter 3370: g_log=[-1.4059227, 0.0060314797, 1.7074966, 0.15625] d_log=[1.7152114, 0.125]
iter 3380: g_log=[-1.4626255, 0.0048154234, 1.7033967, 0.421875] d_log=[1.7245852, 0.21875]
iter 3390: g_log=[-1.4915655, 0.0047151847, 1.7273247, 0.328125] d_log=[1.7403002, 0.328125]
iter 3400: g_log=[-1.4574606, 0.004673155, 1.6911184, 0.453125] d_log=[1.7161516, 0.484375]
iter 3410: g_log=[-1.4758264, 0.005233199, 1.7374864, 0.390625] d_log=[1.6952686, 0.421875]
iter 3420: g_log=[-1.4886191, 0.0052050874, 1.7488735, 0.203125] d_log=[1.7911146, 0.125]
iter 3430: g_log=[-1.479636, 0.00605663, 1.7824675, 0.109375] d_log=[1.7527769, 0.171875]
iter 3440: g_log=[-1.4710064, 0.005064549, 1.7242339, 0.203125] d_log=[1.6545994, 0.328125]
iter 3450: g_log=[-1.5377493, 0.004532954, 1.764397, 0.1875] d_log=[1.7789471, 0.171875]
iter 3460: g_log=[-1.5920963, 0.0046319906, 1.8236959, 0.140625] d_log=[1.7806363, 0.21875]
iter 3470: g_log=[-1.5368452, 0.0050209416, 1.7878923, 0.234375] d_log=[1.7921319, 0.3125]
iter 3480: g_log=[-1.4746476, 0.0058725914, 1.7682772, 0.21875] d_log=[1.8291404, 0.171875]
iter 3490: g_log=[-1.5289599, 0.0043069157, 1.7443056, 0.21875] d_log=[1.7856325, 0.15625]
iter 3500: g_log=[-1.6220096, 0.0049701794, 1.8705186, 0.34375] d_log=[1.8320198, 0.296875]
iter 3510: g_log=[-1.285852, 0.008469474, 1.7093257, 0.234375] d_log=[1.7425511, 0.21875]
iter 3520: g_log=[-1.3820338, 0.0068324097, 1.7236543, 0.265625] d_log=[1.7539017, 0.21875]
iter 3530: g_log=[-1.523902, 0.0057741003, 1.812607, 0.078125] d_log=[1.6918046, 0.34375]
iter 3540: g_log=[-1.5621923, 0.0047767917, 1.801032, 0.1875] d_log=[1.6954322, 0.3125]
iter 3550: g_log=[-1.5542917, 0.0050179344, 1.8051884, 0.15625] d_log=[1.7257781, 0.265625]
iter 3560: g_log=[-1.5179151, 0.0041640974, 1.72612, 0.265625] d_log=[1.7583021, 0.203125]
iter 3570: g_log=[-1.5294809, 0.0043690344, 1.7479327, 0.25] d_log=[1.7856598, 0.171875]
iter 3580: g_log=[-1.5320871, 0.0051070745, 1.7874408, 0.1875] d_log=[1.764884, 0.21875]
iter 3590: g_log=[-1.4958229, 0.0054041683, 1.7660313, 0.203125] d_log=[1.7695799, 0.1875]
iter 3600: g_log=[-1.5037591, 0.0045821443, 1.7328664, 0.15625] d_log=[1.7601807, 0.125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 202us/step - loss: 7.7089 - acc: 0.2006 - val_loss: 7.6518 - val_acc: 0.2723
Epoch 2/30
47401/47401 [==============================] - 5s 105us/step - loss: 7.5823 - acc: 0.2480 - val_loss: 7.4079 - val_acc: 0.2980
Epoch 3/30
47401/47401 [==============================] - 5s 104us/step - loss: 7.4580 - acc: 0.2862 - val_loss: 7.8468 - val_acc: 0.2363
Epoch 4/30
47401/47401 [==============================] - 5s 105us/step - loss: 7.3945 - acc: 0.3166 - val_loss: 7.5465 - val_acc: 0.1842
Epoch 5/30
47401/47401 [==============================] - 5s 105us/step - loss: 7.3181 - acc: 0.3449 - val_loss: 7.2287 - val_acc: 0.4424
Epoch 6/30
47401/47401 [==============================] - 5s 104us/step - loss: 7.2896 - acc: 0.3578 - val_loss: 7.2375 - val_acc: 0.3360
Epoch 7/30
47401/47401 [==============================] - 5s 104us/step - loss: 7.2535 - acc: 0.3754 - val_loss: 7.3513 - val_acc: 0.3250
Epoch 8/30
47401/47401 [==============================] - 5s 105us/step - loss: 7.2289 - acc: 0.3891 - val_loss: 7.1320 - val_acc: 0.4269
Epoch 9/30
47401/47401 [==============================] - 5s 105us/step - loss: 7.1867 - acc: 0.4067 - val_loss: 7.1003 - val_acc: 0.4689
Epoch 10/30
47401/47401 [==============================] - 5s 104us/step - loss: 7.1570 - acc: 0.4163 - val_loss: 7.0942 - val_acc: 0.4763
Epoch 11/30
47401/47401 [==============================] - 5s 104us/step - loss: 7.1648 - acc: 0.4164 - val_loss: 7.2013 - val_acc: 0.4000
Epoch 12/30
47401/47401 [==============================] - 5s 106us/step - loss: 7.1410 - acc: 0.4288 - val_loss: 7.0592 - val_acc: 0.4781
Epoch 13/30
47401/47401 [==============================] - 5s 105us/step - loss: 7.1242 - acc: 0.4341 - val_loss: 7.0611 - val_acc: 0.4520
Epoch 14/30
47401/47401 [==============================] - 5s 106us/step - loss: 7.1041 - acc: 0.4431 - val_loss: 7.0349 - val_acc: 0.4809
Epoch 15/30
47401/47401 [==============================] - 5s 102us/step - loss: 7.1042 - acc: 0.4436 - val_loss: 7.0214 - val_acc: 0.4968
Epoch 16/30
47401/47401 [==============================] - 5s 106us/step - loss: 7.0875 - acc: 0.4530 - val_loss: 7.0930 - val_acc: 0.4323
Epoch 17/30
47401/47401 [==============================] - 5s 106us/step - loss: 7.0632 - acc: 0.4638 - val_loss: 7.1481 - val_acc: 0.4165
Epoch 18/30
47401/47401 [==============================] - 5s 106us/step - loss: 7.0582 - acc: 0.4668 - val_loss: 6.9937 - val_acc: 0.5144
Epoch 19/30
47401/47401 [==============================] - 5s 104us/step - loss: 7.0628 - acc: 0.4643 - val_loss: 7.0429 - val_acc: 0.4962
Epoch 20/30
47401/47401 [==============================] - 5s 103us/step - loss: 5.4351 - acc: 0.4445 - val_loss: 1.5333 - val_acc: 0.4335
Epoch 21/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.3804 - acc: 0.4618 - val_loss: 1.3617 - val_acc: 0.4557
Epoch 22/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.2796 - acc: 0.5137 - val_loss: 1.2050 - val_acc: 0.5640
Epoch 23/30
47401/47401 [==============================] - 5s 105us/step - loss: 1.1820 - acc: 0.5564 - val_loss: 1.1903 - val_acc: 0.5438
Epoch 24/30
47401/47401 [==============================] - 5s 104us/step - loss: 1.1028 - acc: 0.5943 - val_loss: 1.0937 - val_acc: 0.5947
Epoch 25/30
47401/47401 [==============================] - 5s 103us/step - loss: 1.0200 - acc: 0.6312 - val_loss: 0.9716 - val_acc: 0.6591
Epoch 26/30
47401/47401 [==============================] - 5s 101us/step - loss: 0.9490 - acc: 0.6621 - val_loss: 0.9276 - val_acc: 0.6685
Epoch 27/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.8854 - acc: 0.6879 - val_loss: 0.9367 - val_acc: 0.6634
Epoch 28/30
47401/47401 [==============================] - 5s 105us/step - loss: 0.8359 - acc: 0.7059 - val_loss: 1.0594 - val_acc: 0.6310
Epoch 29/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.7932 - acc: 0.7207 - val_loss: 0.8022 - val_acc: 0.7075
Epoch 30/30
47401/47401 [==============================] - 5s 104us/step - loss: 0.7628 - acc: 0.7264 - val_loss: 0.6966 - val_acc: 0.7529
iter 3600: eva_log=eva_acc=0.7263770806585942
iter 3610: g_log=[-1.5345551, 0.0047016987, 1.76964, 0.15625] d_log=[1.7183388, 0.21875]
iter 3620: g_log=[-1.483372, 0.004760202, 1.7213821, 0.3125] d_log=[1.7216125, 0.265625]
iter 3630: g_log=[-1.5073327, 0.004291418, 1.7219036, 0.296875] d_log=[1.7441697, 0.296875]
iter 3640: g_log=[-1.5520743, 0.004249761, 1.7645624, 0.171875] d_log=[1.777321, 0.125]
iter 3650: g_log=[-1.5175198, 0.0048027067, 1.7576551, 0.390625] d_log=[1.7943094, 0.296875]
iter 3660: g_log=[-1.5394524, 0.00446708, 1.7628064, 0.40625] d_log=[1.754809, 0.390625]
iter 3670: g_log=[-1.5197284, 0.0043312507, 1.7362909, 0.265625] d_log=[1.784703, 0.3125]
iter 3680: g_log=[-1.5307285, 0.0039733527, 1.7293961, 0.203125] d_log=[1.7399459, 0.25]
iter 3690: g_log=[-1.5846236, 0.0043063834, 1.7999427, 0.15625] d_log=[1.7085217, 0.265625]
iter 3700: g_log=[-1.5330329, 0.00485258, 1.775662, 0.171875] d_log=[1.7705816, 0.15625]
iter 3710: g_log=[-1.5488588, 0.0053345226, 1.8155849, 0.125] d_log=[1.8020358, 0.203125]
iter 3720: g_log=[-1.5464951, 0.0046806354, 1.7805269, 0.15625] d_log=[1.7904007, 0.15625]
iter 3730: g_log=[-1.5600317, 0.003853759, 1.7527196, 0.203125] d_log=[1.7940116, 0.171875]
iter 3740: g_log=[-1.5300977, 0.0042307293, 1.7416341, 0.125] d_log=[1.7994059, 0.046875]
iter 3750: g_log=[-1.5337794, 0.0046805292, 1.7678058, 0.140625] d_log=[1.7643554, 0.109375]
iter 3760: g_log=[-1.5145799, 0.004485507, 1.7388552, 0.1875] d_log=[1.721122, 0.15625]
iter 3770: g_log=[-1.4717755, 0.0052247243, 1.7330117, 0.234375] d_log=[1.7366552, 0.328125]
iter 3780: g_log=[-1.5084285, 0.005279286, 1.7723927, 0.203125] d_log=[1.7707775, 0.25]
iter 3790: g_log=[-1.4639716, 0.0047756657, 1.7027549, 0.265625] d_log=[1.7915204, 0.140625]
iter 3800: g_log=[-1.4761118, 0.0049885814, 1.7255409, 0.171875] d_log=[1.7319303, 0.1875]
iter 3810: g_log=[-1.4648372, 0.0044498546, 1.6873299, 0.25] d_log=[1.71948, 0.25]
iter 3820: g_log=[-1.5276878, 0.0043911682, 1.7472463, 0.1875] d_log=[1.7229984, 0.265625]
iter 3830: g_log=[-1.5351009, 0.0041366722, 1.7419345, 0.21875] d_log=[1.7002683, 0.296875]
iter 3840: g_log=[-1.4653038, 0.004930038, 1.7118057, 0.234375] d_log=[1.8271606, 0.15625]
iter 3850: g_log=[-1.5766164, 0.003859183, 1.7695756, 0.1875] d_log=[1.7155352, 0.234375]
iter 3860: g_log=[-1.5738593, 0.004140034, 1.780861, 0.171875] d_log=[1.7563373, 0.28125]
iter 3870: g_log=[-1.5051997, 0.0050383583, 1.7571176, 0.171875] d_log=[1.7526671, 0.25]
iter 3880: g_log=[-1.4947305, 0.0043439446, 1.7119277, 0.28125] d_log=[1.746094, 0.203125]
iter 3890: g_log=[-1.506445, 0.004680433, 1.7404667, 0.390625] d_log=[1.7383735, 0.40625]
iter 3900: g_log=[-1.4580566, 0.005070028, 1.711558, 0.265625] d_log=[1.7047185, 0.265625]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 206us/step - loss: 13.9644 - acc: 0.1335 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 2/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 3/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 4/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 5/30
47401/47401 [==============================] - 5s 108us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 6/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 7/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 8/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 9/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 10/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 11/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 12/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 13/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 14/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 15/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 16/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 17/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 18/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 19/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 20/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 21/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 22/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 23/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 24/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 25/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 26/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 27/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 28/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 29/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 30/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
iter 3900: eva_log=eva_acc=0.1315002988758945
iter 3910: g_log=[-1.4978912, 0.004653172, 1.7305498, 0.171875] d_log=[1.7113649, 0.15625]
iter 3920: g_log=[-1.4964478, 0.0044096094, 1.7169282, 0.265625] d_log=[1.744309, 0.234375]
iter 3930: g_log=[-1.5253284, 0.0046583316, 1.758245, 0.203125] d_log=[1.7556733, 0.109375]
iter 3940: g_log=[-1.5275484, 0.004305655, 1.7428312, 0.1875] d_log=[1.7478505, 0.15625]
iter 3950: g_log=[-1.543817, 0.004430041, 1.7653191, 0.34375] d_log=[1.7035935, 0.421875]
iter 3960: g_log=[-1.490749, 0.0051216385, 1.7468309, 0.1875] d_log=[1.7523037, 0.25]
iter 3970: g_log=[-1.5607963, 0.0041828565, 1.7699391, 0.1875] d_log=[1.7819004, 0.15625]
iter 3980: g_log=[-1.5634881, 0.0045854882, 1.7927625, 0.15625] d_log=[1.7910178, 0.140625]
iter 3990: g_log=[-1.518419, 0.0048626, 1.761549, 0.203125] d_log=[1.7617048, 0.21875]
iter 4000: g_log=[-1.5482929, 0.0040665567, 1.7516208, 0.265625] d_log=[1.7304347, 0.28125]
iter 4010: g_log=[-1.547159, 0.0043772543, 1.7660217, 0.234375] d_log=[1.7411553, 0.484375]
iter 4020: g_log=[-1.5176512, 0.0043887785, 1.7370901, 0.234375] d_log=[1.7806362, 0.15625]
iter 4030: g_log=[-1.5029325, 0.004328246, 1.7193449, 0.25] d_log=[1.7359995, 0.1875]
iter 4040: g_log=[-1.5097572, 0.004523756, 1.735945, 0.1875] d_log=[1.6858779, 0.25]
iter 4050: g_log=[-1.4985147, 0.004277872, 1.7124083, 0.21875] d_log=[1.7335567, 0.203125]
iter 4060: g_log=[-1.5627369, 0.003773494, 1.7514116, 0.203125] d_log=[1.7336333, 0.265625]
iter 4070: g_log=[-1.5535836, 0.0041298717, 1.7600772, 0.203125] d_log=[1.762222, 0.25]
iter 4080: g_log=[-1.49626, 0.0048029013, 1.7364051, 0.328125] d_log=[1.7723837, 0.234375]
iter 4090: g_log=[-1.5416434, 0.0046968567, 1.7764862, 0.109375] d_log=[1.7576582, 0.171875]
iter 4100: g_log=[-1.5579607, 0.0045271674, 1.7843192, 0.140625] d_log=[1.763792, 0.421875]
iter 4110: g_log=[-1.4793394, 0.005049332, 1.7318059, 0.375] d_log=[1.7070105, 0.484375]
iter 4120: g_log=[-1.5505852, 0.0037214234, 1.7366563, 0.28125] d_log=[1.7737135, 0.171875]
iter 4130: g_log=[-1.5731701, 0.004006617, 1.7735009, 0.21875] d_log=[1.7388022, 0.203125]
iter 4140: g_log=[-1.5141556, 0.0040139686, 1.714854, 0.265625] d_log=[1.6903565, 0.28125]
iter 4150: g_log=[-1.511703, 0.0042027715, 1.7218416, 0.234375] d_log=[1.7613945, 0.34375]
iter 4160: g_log=[-1.4582431, 0.0051452387, 1.7155051, 0.171875] d_log=[1.7366886, 0.203125]
iter 4170: g_log=[-1.5024698, 0.0042190067, 1.7134202, 0.25] d_log=[1.7580438, 0.140625]
iter 4180: g_log=[-1.5353142, 0.004458732, 1.7582508, 0.109375] d_log=[1.7457969, 0.15625]
iter 4190: g_log=[-1.5554892, 0.003904896, 1.750734, 0.234375] d_log=[1.7989768, 0.09375]
iter 4200: g_log=[-1.5705814, 0.004309548, 1.7860589, 0.0] d_log=[1.7545171, 0.203125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 202us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 2/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 3/30
47401/47401 [==============================] - 5s 103us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 4/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 5/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 6/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 7/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 8/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 9/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 10/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 11/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 12/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 13/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 14/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 15/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 16/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 17/30
47401/47401 [==============================] - 5s 103us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 18/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 19/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 20/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 21/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 22/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 23/30
47401/47401 [==============================] - 5s 103us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 24/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 25/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 26/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 27/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 28/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 29/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
Epoch 30/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.6504 - acc: 0.1531 - val_loss: 13.5862 - val_acc: 0.1571
iter 4200: eva_log=eva_acc=0.1530980359114453
iter 4210: g_log=[-1.5649819, 0.0048053307, 1.8052485, 0.125] d_log=[1.7865167, 0.140625]
iter 4220: g_log=[-1.5481313, 0.0045851795, 1.7773904, 0.25] d_log=[1.7776635, 0.171875]
iter 4230: g_log=[-1.5698502, 0.0044248784, 1.7910941, 0.21875] d_log=[1.804209, 0.203125]
iter 4240: g_log=[-1.5942178, 0.004186696, 1.8035526, 0.140625] d_log=[1.7545409, 0.0]
iter 4250: g_log=[-1.5494261, 0.004521519, 1.7755021, 0.21875] d_log=[1.7819953, 0.171875]
iter 4260: g_log=[-1.4491694, 0.005526688, 1.7255038, 0.1875] d_log=[1.6875, 0.21875]
iter 4270: g_log=[-1.5000774, 0.004406836, 1.7204192, 0.40625] d_log=[1.6945307, 0.421875]
iter 4280: g_log=[-1.5072622, 0.0050607976, 1.7603021, 0.328125] d_log=[1.7025472, 0.453125]
iter 4290: g_log=[-1.5053538, 0.0063240407, 1.8215559, 0.171875] d_log=[1.7366782, 0.234375]
iter 4300: g_log=[-1.544752, 0.0043661622, 1.7630601, 0.296875] d_log=[1.7320871, 0.21875]
iter 4310: g_log=[-1.5089389, 0.0051639, 1.767134, 0.1875] d_log=[1.7364515, 0.234375]
iter 4320: g_log=[-1.4865994, 0.004788542, 1.7260265, 0.28125] d_log=[1.7894173, 0.15625]
iter 4330: g_log=[-1.540312, 0.004593802, 1.7700021, 0.203125] d_log=[1.7626973, 0.171875]
iter 4340: g_log=[-1.5598094, 0.00406562, 1.7630905, 0.1875] d_log=[1.7718028, 0.203125]
iter 4350: g_log=[-1.5705807, 0.0048804907, 1.8146052, 0.140625] d_log=[1.7727182, 0.203125]
iter 4360: g_log=[-1.5122706, 0.0047011366, 1.7473274, 0.21875] d_log=[1.7049258, 0.3125]
iter 4370: g_log=[-1.5944439, 0.0039165574, 1.7902718, 0.171875] d_log=[1.7665172, 0.140625]
iter 4380: g_log=[-1.5989863, 0.004305971, 1.8142848, 0.15625] d_log=[1.8376544, 0.203125]
iter 4390: g_log=[-1.553986, 0.004533858, 1.7806789, 0.265625] d_log=[1.7517278, 0.3125]
iter 4400: g_log=[-1.5387465, 0.0040924535, 1.7433691, 0.21875] d_log=[1.7737591, 0.21875]
iter 4410: g_log=[-1.4747076, 0.00500492, 1.7249537, 0.21875] d_log=[1.7327312, 0.1875]
iter 4420: g_log=[-1.5222219, 0.004310444, 1.7377441, 0.25] d_log=[1.7479308, 0.234375]
iter 4430: g_log=[-1.5122877, 0.0044783065, 1.7362031, 0.140625] d_log=[1.7478213, 0.203125]
iter 4440: g_log=[-1.5266829, 0.0044475314, 1.7490594, 0.375] d_log=[1.7097611, 0.515625]
iter 4450: g_log=[-1.5166912, 0.0049628355, 1.764833, 0.125] d_log=[1.706596, 0.234375]
iter 4460: g_log=[-1.5614043, 0.0040490287, 1.7638558, 0.203125] d_log=[1.7350507, 0.140625]
iter 4470: g_log=[-1.5554967, 0.004305708, 1.7707821, 0.234375] d_log=[1.7737242, 0.234375]
iter 4480: g_log=[-1.5608838, 0.0041527706, 1.7685223, 0.140625] d_log=[1.7604003, 0.25]
iter 4490: g_log=[-1.587639, 0.0037781978, 1.7765489, 0.21875] d_log=[1.7760677, 0.25]
iter 4500: g_log=[-1.5286449, 0.0047996314, 1.7686265, 0.125] d_log=[1.7711183, 0.171875]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 204us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 2/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 3/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 4/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 5/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 6/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 7/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 8/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 9/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 10/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 11/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 12/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 13/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 14/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 15/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 16/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 17/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 18/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 19/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 20/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 21/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 22/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 23/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 24/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 25/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 26/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 27/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 28/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 29/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
Epoch 30/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.5981 - acc: 0.1563 - val_loss: 13.6132 - val_acc: 0.1554
iter 4500: eva_log=eva_acc=0.1554094441355308
iter 4510: g_log=[-1.5379366, 0.004134854, 1.7446792, 0.375] d_log=[1.7399837, 0.125]
iter 4520: g_log=[-1.5384135, 0.0043984456, 1.7583358, 0.15625] d_log=[1.7294006, 0.375]
iter 4530: g_log=[-1.5566695, 0.003923681, 1.7528535, 0.171875] d_log=[1.7054731, 0.234375]
iter 4540: g_log=[-1.5706134, 0.004306794, 1.785953, 0.1875] d_log=[1.713027, 0.234375]
iter 4550: g_log=[-1.5684737, 0.004476428, 1.7922951, 0.09375] d_log=[1.738602, 0.25]
iter 4560: g_log=[-1.511085, 0.0046537267, 1.7437713, 0.203125] d_log=[1.7940204, 0.1875]
iter 4570: g_log=[-1.5068908, 0.004681293, 1.7409555, 0.203125] d_log=[1.6972873, 0.34375]
iter 4580: g_log=[-1.5054321, 0.0041927584, 1.71507, 0.265625] d_log=[1.7564781, 0.265625]
iter 4590: g_log=[-1.5999708, 0.0038662641, 1.793284, 0.203125] d_log=[1.7933007, 0.296875]
iter 4600: g_log=[-1.5757594, 0.004233366, 1.7874277, 0.140625] d_log=[1.791253, 0.171875]
iter 4610: g_log=[-1.530976, 0.004697723, 1.7658622, 0.171875] d_log=[1.7351979, 0.203125]
iter 4620: g_log=[-1.5239704, 0.004357381, 1.7418394, 0.34375] d_log=[1.7116494, 0.4375]
iter 4630: g_log=[-1.5467899, 0.0040852223, 1.751051, 0.140625] d_log=[1.7129924, 0.421875]
iter 4640: g_log=[-1.5398296, 0.003936645, 1.7366619, 0.15625] d_log=[1.7352654, 0.234375]
iter 4650: g_log=[-1.514348, 0.004856999, 1.757198, 0.21875] d_log=[1.6961156, 0.28125]
iter 4660: g_log=[-1.4587035, 0.004969391, 1.7071731, 0.296875] d_log=[1.7585509, 0.203125]
iter 4670: g_log=[-1.5338825, 0.0046990807, 1.7688365, 0.1875] d_log=[1.7475308, 0.296875]
iter 4680: g_log=[-1.5850608, 0.0043892777, 1.8045247, 0.28125] d_log=[1.7972643, 0.09375]
iter 4690: g_log=[-1.5564226, 0.0040850276, 1.760674, 0.125] d_log=[1.76775, 0.4375]
iter 4700: g_log=[-1.5611793, 0.0042258445, 1.7724715, 0.1875] d_log=[1.7738618, 0.34375]
iter 4710: g_log=[-1.523792, 0.004494739, 1.748529, 0.15625] d_log=[1.7698613, 0.21875]
iter 4720: g_log=[-1.4656804, 0.005197131, 1.725537, 0.234375] d_log=[1.7516209, 0.140625]
iter 4730: g_log=[-1.4862682, 0.0049838615, 1.7354612, 0.40625] d_log=[1.7426506, 0.21875]
iter 4740: g_log=[-1.5762252, 0.0037959605, 1.7660232, 0.15625] d_log=[1.7511048, 0.21875]
iter 4750: g_log=[-1.5723014, 0.003752422, 1.7599225, 0.1875] d_log=[1.7410905, 0.28125]
iter 4760: g_log=[-1.5783257, 0.0040287827, 1.7797649, 0.140625] d_log=[1.7707276, 0.234375]
iter 4770: g_log=[-1.5508755, 0.0040748185, 1.7546165, 0.390625] d_log=[1.7589964, 0.390625]
iter 4780: g_log=[-1.5925267, 0.0035318248, 1.769118, 0.1875] d_log=[1.768456, 0.1875]
iter 4790: g_log=[-1.5560441, 0.003863381, 1.7492131, 0.265625] d_log=[1.7457147, 0.234375]
iter 4800: g_log=[-1.5872535, 0.0039211814, 1.7833126, 0.078125] d_log=[1.7557262, 0.453125]
Train on 47401 samples, validate on 8365 samples
Epoch 1/30
47401/47401 [==============================] - 10s 203us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 2/30
47401/47401 [==============================] - 5s 103us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 3/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 4/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 5/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 6/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 7/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 8/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 9/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 10/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 11/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 12/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 13/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 14/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 15/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 16/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 17/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 18/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 19/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 20/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 21/30
47401/47401 [==============================] - 5s 104us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 22/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 23/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 24/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 25/30
47401/47401 [==============================] - 5s 105us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 26/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 27/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 28/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 29/30
47401/47401 [==============================] - 5s 107us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
Epoch 30/30
47401/47401 [==============================] - 5s 106us/step - loss: 13.9694 - acc: 0.1333 - val_loss: 13.9986 - val_acc: 0.1315
iter 4800: eva_log=eva_acc=0.1315002988758945
iter 4810: g_log=[-1.5387015, 0.0038496535, 1.7311842, 0.21875] d_log=[1.7771665, 0.21875]
iter 4820: g_log=[-1.5923067, 0.0043108985, 1.8078517, 0.125] d_log=[1.8210533, 0.109375]
iter 4830: g_log=[-1.5314994, 0.004671827, 1.7650907, 0.265625] d_log=[1.7919183, 0.15625]
iter 4840: g_log=[-1.4808522, 0.005383349, 1.7500197, 0.21875] d_log=[1.743393, 0.1875]
iter 4850: g_log=[-1.5452042, 0.0036247144, 1.7264398, 0.234375] d_log=[1.7426468, 0.140625]
iter 4860: g_log=[-1.4980093, 0.0044265245, 1.7193356, 0.375] d_log=[1.7136534, 0.421875]
iter 4870: g_log=[-1.4280285, 0.0053005703, 1.693057, 0.34375] d_log=[1.7471833, 0.15625]
iter 4880: g_log=[-1.5278479, 0.0044394303, 1.7498194, 0.40625] d_log=[1.7276328, 0.3125]
iter 4890: g_log=[-1.5486114, 0.0037605893, 1.7366409, 0.1875] d_log=[1.7450886, 0.296875]
iter 4900: g_log=[-1.5592194, 0.004022393, 1.760339, 0.21875] d_log=[1.789928, 0.21875]
iter 4910: g_log=[-1.5926946, 0.0037591215, 1.7806507, 0.171875] d_log=[1.7857366, 0.1875]
iter 4920: g_log=[-1.5804901, 0.0036214483, 1.7615626, 0.265625] d_log=[1.7582005, 0.21875]
iter 4930: g_log=[-1.4881033, 0.0035972719, 1.6679668, 0.375] d_log=[1.7701819, 0.125]
iter 4940: g_log=[-1.5419023, 0.0056629395, 1.8250493, 0.296875] d_log=[1.7868731, 0.203125]
iter 4950: g_log=[-1.511678, 0.0045164838, 1.7375022, 0.21875] d_log=[1.7237971, 0.21875]
iter 4960: g_log=[-1.5279645, 0.0041381544, 1.7348722, 0.234375] d_log=[1.7071486, 0.28125]
iter 4970: g_log=[-1.5619606, 0.0040492266, 1.7644219, 0.203125] d_log=[1.7648022, 0.21875]
iter 4980: g_log=[-1.565162, 0.003883273, 1.7593256, 0.21875] d_log=[1.7756821, 0.21875]
iter 4990: g_log=[-1.5250623, 0.004027537, 1.7264392, 0.28125] d_log=[1.7529109, 0.1875]

